# <center>Ch3 企业级完全分布式_多机多卡金融领域私有数据GRPO微调</center>

# 1. 为什么需要GRPO微调

## 1. 强化学习训练阶段的必要性分析

### 1.1 SFT的根本局限性

**SFT的本质：**
- **训练方式：** 基于固定的输入-输出对进行监督学习
- **学习目标：** 最大化给定输入下输出正确答案的概率

**SFT的理论局限：**

**1. 静态优化目标**
- SFT只能学习到训练数据中的固定映射关系
- 无法处理同一问题的多种合理回答
- 缺乏对回答质量的动态评估机制

**2. 数据标注的主观性问题**
- 人类标注者对"好答案"的定义存在主观差异
- 不同标注者可能给出不同的"标准答案"
- 静态的标注无法捕捉复杂的人类偏好

**3. 探索能力的缺失**
- SFT是纯粹的利用，没有探索
- 模型只能学习到训练数据覆盖的解决方案
- 无法发现可能更优的新策略

### 1.2 人类偏好的复杂性

**偏好的多维性：**
- **内容质量：** 准确性、完整性、深度
- **表达方式：** 清晰度、逻辑性、可读性
- **价值对齐：** 安全性、伦理性、有用性

**偏好的动态性：**
- 同一用户在不同场景下的偏好可能不同
- 社会价值观和标准会随时间演变
- 领域专业知识的发展会影响质量标准

**偏好的相对性：**
- 人类更容易进行相对比较而非绝对评分
- "A比B好"的判断比"A的质量是8分"更可靠
- 这正是强化学习中奖励信号的优势

## 2. 强化学习在LLM中的核心作用

### 2.1 理论基础：奖励建模

**奖励模型的引入：**
- **目的：** 将复杂的人类偏好转化为可计算的奖励信号
- **方法：** 通过人类比较数据训练奖励模型

**奖励模型的优势：**
- 可以处理主观和复杂的评价标准
- 能够泛化到未见过的输入-输出对
- 提供连续的质量评估而非离散的标签

### 2.2 策略优化的理论意义

**策略梯度方法的核心：**
```
∇θ J(θ) = E[∇θ log π(a|s) · A(s,a)]
```
- **π(a|s)：** 在状态s下选择动作a的概率（模型生成概率）
- **A(s,a)：** 优势函数，衡量动作a相对于平均水平的优势
- **目标：** 增加高奖励动作的概率，减少低奖励动作的概率

**相比SFT的优势：**
- **动态优化：** 根据奖励信号动态调整策略
- **探索机制：** 通过随机性探索新的解决方案
- **质量导向：** 直接优化最终目标（奖励）而非中间目标（匹配训练数据）

### 2.3 对齐问题的解决

**对齐的三个层面：**

**1. 有用性对齐**
- 模型输出应该对用户有实际帮助
- 强化学习可以通过奖励信号强化有用行为

**2. 安全性对齐**
- 模型不应输出有害或危险内容
- 通过负奖励惩罚不当行为

**3. 诚实性对齐**
- 模型应该承认不知道而非编造信息
- 通过奖励诚实回答来实现

## 3. SFT是否足够的理论分析

### 3.1 理论上的可能性

**完美SFT的假设：**
如果能够构建一个包含所有可能输入和所有最优输出的完美数据集，理论上SFT可能足够。

**理论限制：**
- **数据集大小：** 需要指数级别的数据量
- **标注一致性：** 需要完全一致的质量标准
- **覆盖完整性：** 需要覆盖所有可能的输入空间

### 3.2 实际的不可行性

**组合爆炸问题：**
- 自然语言的组合空间是指数级的
- 即使是有限的输入空间，最优输出也可能有多个
- 构建完美数据集在计算上是不可行的

**标注成本问题：**
- 高质量标注需要领域专家
- 标注成本随数据规模线性增长
- 质量控制需要多轮验证

**动态性问题：**
- 人类偏好会随时间变化
- 新的应用场景不断出现
- 静态数据集无法适应动态需求


## 4. GRPO技术原理分析

### 4.1 传统PPO的核心问题

**PPO的双网络架构限制：**
- **Actor网络**：策略网络π(a|s)，负责生成动作
- **Critic网络**：价值网络V(s)，负责评估状态价值
- **计算开销**：需要同时训练两个网络，内存和计算资源翻倍
- **训练复杂性**：两个网络的训练需要协调，容易出现不稳定

### 4.2 GRPO的核心创新原理

**组内相对比较机制：**
GRPO的根本创新在于**放弃绝对价值评估，转向相对价值比较**。

**理论基础：**
- **相对评估的稳定性**：人类更容易做相对比较而非绝对评分
- **组内归一化**：通过组内比较消除全局奖励尺度的影响
- **基线自适应**：基线从数据中自动学习，无需额外网络

**数学原理：**
```json
传统方法：需要学习 V(s) 来估计状态价值
GRPO方法：直接从组内分数 {r_1, r_2, ..., r_G} 计算相对优势
```


### 4.3 组内基线估计机制

**基线估计的数学原理：**
- **中心化处理**：通过减去组内均值实现中心化
- **方差减少**：组内比较减少了全局奖励分布的方差影响
- **自适应性**：基线随着组内样本质量自动调整


## 5. GRPO的优势

### 5.1 计算效率优势

**内存使用量对比：**
```json
PPO: Actor网络参数 + Critic网络参数 ≈ 2 × 模型参数
GRPO: Actor网络参数 ≈ 1 × 模型参数
内存节省：约50%
```

**计算复杂度对比：**
```json
PPO每步计算：
- 前向传播：Actor + Critic
- 反向传播：Actor + Critic
- 总计算量：2 × 单网络计算量

GRPO每步计算：
- 前向传播：仅Actor
- 反向传播：仅Actor
- 总计算量：1 × 单网络计算量
```



### 5.2 训练稳定性优势

**避免Actor-Critic不匹配：**
- **PPO问题**：Actor和Critic的训练步调不一致可能导致不稳定
- **GRPO优势**：单网络训练，避免了网络间的不匹配问题

**组内归一化的稳定性：**
- **奖励尺度不敏感**：组内比较自动消除了全局奖励尺度的影响
- **自适应基线**：基线随数据分布自动调整
- **减少方差**：组内比较减少了奖励估计的方差

### 5.3 样本效率优势

**组内学习的效率：**
```json
传统方法：每个样本独立学习
GRPO方法：G个样本组内互相学习，信息利用率更高
```

**相对比较的优势：**
- **判别性更强**：相对比较比绝对评分更准确
- **噪声抑制**：组内比较可以抑制奖励模型的噪声
- **泛化能力**：相对关系比绝对分数更容易泛化

# 2. 支持GRPO的架构

### 1. **HuggingFace TRL + DeepSpeed**（主流推荐）

**技术栈：**
- **TRL (Transformer Reinforcement Learning)** - HuggingFace的官方强化学习库
- **DeepSpeed/Accelerate** - 分布式训练支持
- **PyTorch** 作为底层框架

**详细原因：**
- **官方支持**：TRL是HuggingFace官方维护的库，有专门的`GRPOTrainer`
- **成熟生态**：完整集成了PEFT、vLLM、Liger等优化技术
- **真实实现**：这是DeepSeek团队用于训练DeepSeek-R1的相同算法实现
- **分布式能力**：原生支持多机多卡训练，通过Accelerate和DeepSpeed

**适用场景：**
- 各种规模的模型训练（0.5B到70B+）
- 需要PEFT优化的场景
- 需要vLLM加速推理的环境
- 标准的学术/工业研究环境

### 2. **OpenRLHF + Ray** (实验性选择)

**技术栈：**
- 基于Ray/DeepSpeed的分布式实现
- 支持多种RL算法包括GRPO

**详细原因：**
- **灵活性**：支持多种RL算法对比
- **云原生**：基于Ray的分布式架构
- **实验性**：相对较新，社区支持有限

**适用场景：**
- 需要对比多种RL算法
- 有Ray使用经验的团队
- 大规模云环境部署



## **总结对比**

| 框架 | 成熟度 | GRPO实现 | 分布式后端 | 多机多卡支持 | 推荐指数 |
|------|--------|----------|------------|-------------|----------|
| **HuggingFace TRL** | 最高 | 官方原生 | DeepSpeed/Accelerate | ✅ | ⭐⭐⭐⭐⭐ |
| **OpenRLHF** | 高 | 独立实现 | Ray | ✅ | ⭐⭐⭐⭐ |
| **Colossal-AI** | 中等 | 自实现 | 自有系统 | ✅ | ⭐⭐ |
| **其他框架** | 低 | 无/有限 | 各自不同 | 部分支持 | ⭐ |



### **1. GRPO实现方式的区别**

| 框架 | GRPO实现方式 | 架构基础 |
|------|-------------|----------|
| **HuggingFace TRL** | 原生GRPOTrainer | 专门的GRPO实现 |
| **OpenRLHF** | PPOTrainer + group_norm参数 | PPO框架的变种 |

### **2. 分布式策略的区别**

#### **HuggingFace TRL + DeepSpeed**
- 单一模型的分布式训练
- DeepSpeed ZeRO自动处理模型分片
- 适合标准的GRPO训练流程

#### **OpenRLHF + DeepSpeed（无Ray）**
- 也是单一模型分布式训练
- 与HuggingFace方案技术上相似
- 但GRPO实现不如TRL原生

#### **OpenRLHF + Ray + DeepSpeed**
- 多模型分离架构（Actor/Critic/Reward分离）
- Ray负责调度，DeepSpeed负责每个模型内部分片
- 适合复杂的RLHF场景，但对纯GRPO可能过度设计


其他框架推荐   https://github.com/hiyouga/EasyR1

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250708200251616.png" width=100%></div>

# 3. 数据准备

## 1. 核心理论基础与实践应用框架

### 1.1 信息论指导的实践原则

**理论基础：**
互信息最大化原理 I(P;Q) = H(Q) - H(Q|P)，其中P是prompt，Q是回答质量

**为什么要最大化 I(P;Q)？**

1. **P（Prompt）的作用：** 提供模型生成高质量回答的信息
2. **Q（回答质量）的目标：** 我们希望生成的回答质量高
3. **互信息的意义：** I(P;Q)越大，说明prompt对回答质量的影响越大

**数学推导的实际意义：**
```json
I(P;Q) = H(Q) - H(Q|P)
```

- **H(Q)：** 回答质量的总体不确定性（固定值）
- **H(Q|P)：** 给定prompt后，回答质量的剩余不确定性
- **目标：** 最小化H(Q|P)，即给定prompt后，回答质量应该尽可能确定

**实践转化：**
- **低H(Q|P)意味着：** 好的prompt能够较确定地引导出高质量回答
- **具体要求：** prompt必须包含足够信息，减少回答的随机性
- **质量标准：** 同一个高质量prompt，不同模型应该生成相似质量的回答



### 1.2 信息论指导的数据设计

**信息完整性量化：**
```json
完整性得分 = 1 - H(Q|P) / H(Q)
```
- **得分范围：** 0到1，越接近1表示prompt信息越完整
- **实际含义：** 衡量prompt对回答质量的决定程度


**实践应用：**

**信息完整性原则**
- **理论要求：** 降低条件熵H(Q|P)，确保给定prompt后回答质量的确定性
- **实践操作：** 每个prompt必须包含以下核心信息元素：
  - 明确的任务背景（30-40%信息量）
  - 具体的数据或事实（40-50%信息量）
  - 清晰的问题导向（10-20%信息量）


**实践应用实例：**
```json
低完整性prompt：
"分析股票走势" 
-> H(Q|P)很大，因为缺乏具体信息

高完整性prompt：
"苹果公司2024年Q1营收1195亿美元，同比增长2%，iPhone销售额690亿美元，同比下降15%。服务业务收入231亿美元，同比增长11%。基于这些数据，分析苹果公司的业务结构变化和未来增长潜力。"
-> H(Q|P)较小，具体信息限制了回答的随机性
```


### 1.2 认知负荷理论的实践转化

**理论基础：**
Sweller的认知负荷理论，区分内在、外在和相关认知负荷

**核心假设：**
1. **工作记忆容量有限：** 同时处理的信息元素数量受限（约7±2个）
2. **长期记忆容量无限：** 但检索需要通过工作记忆
3. **学习发生在工作记忆中：** 新信息与已有知识整合

**三种认知负荷详解：**

**内在认知负荷（Intrinsic Cognitive Load）**
- **定义：** 任务本身的复杂性产生的认知负荷
- **特点：** 由学习材料的元素交互性决定，不可减少
- **实际例子：** 
  - 低内在负荷：记忆单个单词
  - 高内在负荷：理解复杂的数学证明

**外在认知负荷（Extraneous Cognitive Load）**
- **定义：** 由不当的教学设计产生的额外负荷
- **特点：** 可以通过优化设计减少或消除
- **实际例子：**
  - 高外在负荷：信息分散、格式混乱
  - 低外在负荷：结构清晰、重点突出

**相关认知负荷（Germane Cognitive Load）**
- **定义：** 用于处理、构建和自动化图式的认知负荷
- **特点：** 有利于学习，应该被最大化
- **实际例子：**
  - 深度思考和理解
  - 知识整合和应用


**为什么认知负荷理论支撑GRPO数据设计？**

**类比关系：**
- **人类学习 → 模型学习**
- **工作记忆 → 模型的上下文窗口**
- **认知负荷 → 模型的处理复杂度**

**应用策略：**

**控制内在负荷：**
```json
任务复杂度 = 概念数量 × 概念间关系复杂度
```
- **简单任务：** 3-5个概念，线性关系
- **中等任务：** 5-8个概念，部分交互
- **复杂任务：** 8+个概念，多重交互

**最小化外在负荷：**
- **统一格式：** 所有prompt使用相同的结构模板
- **清晰组织：** 信息按逻辑顺序呈现
- **避免冗余：** 去除不必要的修饰词和重复信息

**最大化相关负荷：**
- **促进思考：** 设计需要推理和整合的任务
- **建立联系：** 连接新信息与已有知识
- **深度处理：** 要求分析、评估和创造

**实践应用：**

**认知负荷优化策略**
- **内在负荷控制：** 根据模型能力设置适当的任务复杂度
- **外在负荷最小化：** 统一prompt结构，减少格式理解负担
- **相关负荷最大化：** 设计促进深度学习的认知挑战

**实际操作模板：**
```json
标准prompt结构：
[背景信息块] + [关键数据块] + [任务导向块] + [约束条件块]

认知复杂度层次：
- 初级：事实理解和简单分析（占比30%）
- 中级：关系推理和比较评估（占比50%）
- 高级：创新思维和复杂决策（占比20%）
```


## 2. 系统化数据准备实施方案

### 2.1 三层数据架构设计

**理论支撑：**
基于统计学习理论的偏差-方差权衡和样本复杂度理论

- **含义：** 模型预测值与真实值的系统性差异
- **产生原因：** 模型假设过于简单，无法捕捉真实关系
- **实际表现：** 欠拟合，在训练集和测试集上都表现不佳

**偏差控制策略：**

**问题：** 数据集不够代表性导致系统性偏差    
**解决方案：**
- **全面覆盖：** 确保数据涵盖所有重要的任务类型和难度层级
- **领域平衡：** 不同子领域的数据比例要合理
- **质量一致：** 统一的质量标准避免系统性质量偏差

**方差控制策略：**

**问题：** 训练数据的微小变化导致模型性能大幅波动    
**解决方案：**
- **数据增强：** 通过多样化的表述方式增加数据多样性
- **质量标准化：** 统一的评估标准和处理流程
- **交叉验证：** 多次验证确保结果的稳定性

**噪声控制策略：**

**问题：** 数据中的错误和不一致性    
**解决方案：**
- **多重验证：** 多个评估者独立评估
- **一致性检查：** 自动检测和修正不一致的标注
- **质量门槛：** 设置最低质量标准

**架构：**

**第一层：核心种子数据（Core Seed Data）**
- **规模：** 总量的10-15%
- **质量要求：** 人工精选和验证
- **理论依据：** 高质量种子数据降低偏差，提供学习基准
- **实际操作：**
  - 从现有SFT数据中提取最高质量的prompt
  - 领域专家人工评估和优化
  - 建立质量标准和评估基准

**第二层：扩展生成数据（Expanded Generated Data）**
- **规模：** 总量的60-70%
- **生成策略：** 基于种子数据的变体生成
- **理论依据：** 通过数据增强降低方差，提高泛化能力
- **实际操作：**
  - 使用高质量LLM（如GPT-4）基于种子数据生成变体
  - 保持核心信息结构，变化表述方式和具体细节
  - 自动化质量筛选和人工抽样验证

**第三层：领域特化数据（Domain-Specific Data）**
- **规模：** 总量的15-25%
- **特化方向：** 针对特定领域或任务类型
- **理论依据：** 领域特化提高模型在特定场景的性能
- **实际操作：**
  - 识别关键应用场景和高价值任务
  - 设计专门的prompt模板和生成策略
  - 与领域专家合作确保专业性


### 2.2 实施流程与时间粗略规划

**阶段1：基础建设期（2-3周）**
- **理论准备：** 建立评估标准和质量基准
- **实践任务：**
  - 建立数据处理和评估pipeline
  - 训练评估人员，统一质量标准
  - 准备种子数据集（500-1000条）
- **质量检查：** 人工全量验证，建立baseline

**阶段2：扩展生成期（4-6周）**
- **理论指导：** 基于种子数据的系统化扩展
- **实践任务：**
  - 大规模生成扩展数据（5000-10000条）
  - 实施自动化质量筛选
  - 人工抽样验证（10-20%抽样率）
- **质量检查：** 多维度质量评估，持续优化

**阶段3：优化完善期（2-3周）**
- **理论验证：** 通过小规模训练验证数据质量
- **实践任务：**
  - 基于训练反馈优化数据集
  - 补充特定类型的数据
  - 最终质量验证和文档化
- **质量检查：** 全面质量审计，形成最终数据集

## 3. 关键技术实现方案

### 3.1 基于LLM的高质量数据生成

**理论基础：**
上下文学习理论和prompt工程原理

**实践方案：**

**多层次prompt设计**
```json
    System Prompt层：
    定义生成任务的基本要求和约束条件

    Context Prompt层：
    提供领域知识和示例模板

    Task Prompt层：
    具体的生成指令和质量要求

    Constraint Prompt层：
    格式要求和避免事项
```

**质量控制策略**
- **多模型验证：** 使用2-3个不同的高质量模型生成，对比一致性
- **迭代优化：** 基于质量评估结果不断调整prompt
- **人工介入：** 在关键节点进行人工审核和调整

### 3.2 人机协作的质量保证机制

**理论基础：**
人机协作理论和质量管理原理

**实践框架：**

**人工智能协作模式**
- **AI负责：** 大规模数据生成、初步质量筛选、统计分析
- **人工负责：** 质量标准制定、关键样本验证、最终审核
- **协作机制：** AI提供候选，人工进行决策和优化

**质量保证流程**
1. **自动生成**：AI批量生成候选数据
2. **智能筛选**：多维度自动评估和排序
3. **人工验证**：专家验证高质量候选
4. **反馈优化**：基于验证结果调整生成策略
5. **最终审核**：全面质量检查和文档化



## 4. 项目实施的关键成功要素

### 4.1 风险管理与应对策略

**理论基础：**
项目风险管理理论和不确定性处理

**主要风险与应对：**

**技术风险**
- **风险描述：** 生成质量不稳定，自动评估不准确
- **应对策略：** 建立多套备选方案，人工介入机制
- **监控指标：** 质量得分分布，异常数据比例

**资源风险**
- **风险描述：** 计算资源不足，时间进度延误
- **应对策略：** 资源提前规划，分阶段实施
- **监控指标：** 资源使用率，进度完成率

**质量风险**
- **风险描述：** 数据质量达不到要求，影响训练效果
- **应对策略：** 严格质量标准，多重验证机制
- **监控指标：** 质量评估分数，训练效果反馈

### 4.2 持续改进与优化机制

**理论基础：**
PDCA循环和持续改进理论

**四个阶段：**

**Plan（计划）：**
- **活动：** 识别问题，制定改进计划
- **输出：** 具体的行动方案和成功指标
- **在GRPO中：** 分析数据质量问题，制定优化策略

**Do（执行）：**
- **活动：** 实施计划，收集数据
- **输出：** 执行结果和过程数据
- **在GRPO中：** 实施数据优化方案，生成新数据

**Check（检查）：**
- **活动：** 评估结果，分析偏差
- **输出：** 分析报告和改进建议
- **在GRPO中：** 评估数据质量改进效果

**Act（行动）：**
- **活动：** 标准化成功做法，计划下一轮改进
- **输出：** 更新的标准和新的改进目标
- **在GRPO中：** 固化有效的数据处理方法

**实践机制：**

**反馈循环系统**
- **数据收集：** 训练过程中的性能指标和问题反馈
- **分析诊断：** 识别数据质量问题和改进机会
- **优化实施：** 基于分析结果调整数据准备策略
- **效果验证：** 通过后续训练验证改进效果

**版本管理策略**
- **数据版本化：** 每次重要更新都创建新版本
- **变更记录：** 详细记录每次变更的原因和影响
- **回滚机制：** 出现问题时能够快速回滚到稳定版本

## 5. 实际应用指导

### 5.1 金融领域专项实施方案

**基于项目背景的具体应用**

**金融prompt设计原则**
- **信息要素：** 市场数据、财务指标、时间背景、影响因素
- **认知要求：** 数据分析、风险评估、趋势预测、决策建议
- **开放性控制：** 允许多种分析角度，但保持专业性约束

**质量标准定制**
- **专业性要求：** 使用准确的金融术语和概念
- **时效性要求：** 反映最新的市场情况和监管要求
- **实用性要求：** 生成的分析应该具有实际指导价值


# 4. 环境配置

## 多机多卡配置

主节点 10.60.68.220     
从节点 10.60.98.173


数据挂载(安装NFS)    
主节点操作

sudo apt install nfs-kernel-server -y


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250704181742844.png" width=100%></div>


```bash
#配置NFS导出（使用内网网段）
sudo tee /etc/exports << EOF
/shared 10.60.0.0/16(rw,sync,no_subtree_check,no_root_squash,insecure)
EOF
#启动NFS服务
sudo systemctl enable nfs-kernel-server
sudo systemctl start nfs-kernel-server
sudo exportfs -ra
 #验证NFS配置
sudo exportfs -v
showmount -e localhost
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250704185826273.png" width=100%></div>



```bash
# 从节点
sudo apt install nfs-common -y
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250704182037002.png" width=100%></div>


```bash
#测试挂载
sudo mount -t nfs 10.60.68.220:/shared /shared
 #验证挂载
ls -la /shared/
df -h | grep shared 
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250704185914785.png" width=100%></div>


```bash
# 设置开机启动挂载(注意开机时候主节点先开机)
echo "10.60.98.173:/shared /shared nfs defaults,_netdev 0 0" | sudo tee -a /etc/fstab
```


### 节点之间免密登录


```bash
# 服务器之间相互免密登录
#在主节点上执行 
ssh-keygen -t rsa -b 4096 -f ~/.ssh/id_rsa -N "" 
 #查看生成的公钥
cat ~/.ssh/id_rsa.pub
```


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250704200042336.png" width=100%></div>


```bash
ssh-copy-id -o StrictHostKeyChecking=no ubuntu@10.60.98.173
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250704200312123.png" width=100%></div>


```bash
# 测试免密是否成功
ssh ubuntu@10.60.68.220 "hostname && whoami"
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250704200403292.png" width=100%></div>


```bash
# 从节点也做免密登录
cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250704200623058.png" width=100%></div>


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250704213123304.png" width=100%></div>


### 安装pdsh

```bash
# 主节点执行
sudo apt install pdsh 
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250704203553257.png" width=100%></div>


```bash
# 设置pdsh执行方式为ssh
export PDSH_RCMD_TYPE=ssh
# 测试
pdsh -R ssh -w 10.60.68.220,10.60.98.173 "hostname"

# 如果针对本机执行需要输出秘钥直接将秘钥追加到本机即可
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250704213137089.png" width=100%></div>


# 5. GRPO微调实战

## 环境配置

```bash
conda create -n reward3 python=3.11 -y
pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1  -i https://pypi.tuna.tsinghua.edu.cn/simple
pip install accelerate==0.34.2 -i https://pypi.tuna.tsinghua.edu.cn/simple
pip install transformers==4.51.3 -i https://pypi.tuna.tsinghua.edu.cn/simple
pip install deepspeed==0.15.4 -i https://pypi.tuna.tsinghua.edu.cn/simple
pip install datasets==3.3.1 -i https://pypi.tuna.tsinghua.edu.cn/simple
pip install TRL==0.18.2 -i https://pypi.tuna.tsinghua.edu.cn/simple
pip install peft -i https://pypi.tuna.tsinghua.edu.cn/simple
pip install liger-kernel -i https://pypi.tuna.tsinghua.edu.cn/simple
pip install ninja -i https://pypi.tuna.tsinghua.edu.cn/simple
sudo apt install ninja-build
pip install tensorboard -i https://pypi.tuna.tsinghua.edu.cn/simple
```

## 项目讲解

### 一、项目技术定位与架构概述

#### 1.1 技术栈定位
本项目是基于DeepSeek-R1-Distill-Qwen-7B的金融领域GRPO强化学习微调系统，采用企业级分布式训练架构。核心技术栈为HuggingFace TRL + DeepSpeed + PyTorch，支持多机多卡生产环境部署。

#### 1.2 系统架构层次
```json
业务层: 金融问答模型微调
算法层: GRPO强化学习训练引擎  
框架层: TRL训练器 + DeepSpeed分布式
基础层: A800集群 + NCCL通信
```
####  项目目录结构分析

```bash
grpo_financial_tuning/          #  核心训练系统
├── configs/                    #  配置管理
│   ├── config.json            # 主配置文件
│   └── deepspeed_zero2.json   # DeepSpeed分布式配置
├── data/                       #  数据处理模块
│   ├── filtered_financial_news_5k.jsonl  # 原始英文金融数据
│   └── grpo_prompt.py          # 数据转换脚本
├── utils/                      #  工具模块
│   ├── config_loader.py        # 配置加载器
│   ├── dataset.py              # 数据集处理
│   └── logger.py               # 日志管理
├── model_eval/                 #  评估模块
│   └── multigpu_evaluator.py   # 多GPU评估工具
├── train.py                    #  主训练脚本
├── test.py                     #  模型测试脚本
├── run_training.sh             #  训练启动脚本
├── run_evaluation.sh           #  评估启动脚本
└── deploy_A800.sh              #  A800部署脚本
```


### 二、核心模块架构设计

#### 2.1 训练引擎架构 (train.py)

**设计模式**: 组件化初始化 + 回调机制
- **GRPOTraining主类**: 统一管理训练生命周期，采用依赖注入模式
- **组件初始化顺序**: tokenizer → model → dataset → reward_function → trainer
- **早停回调机制**: GRPOBestModelEarlyStoppingCallback实现智能模型保存

**关键技术决策**:
- 内存优化策略: device_map=None让DeepSpeed完全控制设备分配
- 梯度管理: 强制设置requires_grad=True确保参数可训练

**扩展性设计**:
- 配置驱动: 所有超参数通过ProjectConfig统一管理
- 模型适配: 支持LoRA和全参数微调切换
- 检查点恢复: 原生支持训练中断恢复

#### 2.2 数据处理架构 (data/grpo_prompt.py)

**并发处理设计**:
- **线程池架构**: ThreadPoolExecutor管理工作线程
- **流控策略**: 随机延迟分散API请求时间

**数据转换流水线**:
```json
英文金融新闻 → 文本截断 → DeepSeek API → JSON解析 → 中文问答格式
```

**质量保证机制**:
- 分层随机采样确保数据代表性
- JSON格式验证和错误处理
- 生成结果示例展示和统计报告

#### 2.3 分布式配置架构 (configs/)

**配置分层设计**:
- **业务配置层**: config.json管理模型路径、训练参数、GRPO参数
- **框架配置层**: deepspeed_zero2.json管理分布式优化参数
- **运行时配置**: 命令行参数支持动态覆盖

**DeepSpeed优化策略**:
- ZeRO-2阶段: 优化器状态CPU卸载 + 梯度分片
- 混合精度: bf16降低内存占用同时保持数值稳定性

### 三、分布式训练架构

#### 3.1 多机多卡部署架构

**集群拓扑设计**:
- 主节点(10.60.68.220): 2×A800-80GB + 训练协调
- 从节点(10.60.98.173): 1×A800-80GB + 计算节点

**通信架构优化**:
- NCCL通信库: 专门优化GPU间高带宽通信
- 网络配置: 强制使用eth0网卡，禁用InfiniBand避免兼容性问题
- 缓冲区调优: NCCL_BUFFSIZE=524288统一缓冲区大小

**进程管理策略**:
- DeepSpeed launcher: 自动处理多机进程启动和同步
- hostfile配置: 声明式定义集群拓扑和GPU分配
- 故障恢复: 支持节点故障检测和训练恢复

#### 3.2 内存管理架构

**ZeRO-2内存优化**:
- 优化器状态分片: 每个GPU只保存部分优化器状态
- CPU卸载策略: 优化器状态存储在CPU内存，训练时动态加载
- 梯度分片: 反向传播时梯度按需聚合，降低峰值内存

**显存分配策略**:
- 模型权重: 每个GPU保存完整副本用于前向传播
- 激活值管理: 梯度检查点技术平衡计算和内存
- 动态内存池: PyTorch CUDA内存池优化小块内存分配

### 四、GRPO算法集成架构

#### 4.1 奖励模型集成

**模型加载架构**:
- 独立奖励模型: Skywork-Reward-checkpoint-1000
- 格式对齐: LLaMA-3特殊token格式确保与训练时一致
- 设备管理: 奖励模型独立设备分配避免显存冲突

**奖励计算流水线**:
```
对话格式化 → 文本编码 → 奖励模型推理 → 分数提取 → 异常处理
```

#### 4.2 GRPO训练流程

**生成阶段**:
- 多样本生成: num_generations=4为每个prompt生成多个回复
- 采样策略: temperature=0.7 + top_p=0.9平衡质量和多样性
- 长度控制: max_completion_length=1024限制生成长度

**优化阶段**:
- 组内比较: 4个生成样本组内相对评分
- 策略更新: epsilon=0.2控制策略更新幅度
- 梯度裁剪: max_grad_norm=1.0防止梯度爆炸

### 五、系统可扩展性架构

#### 5.1 水平扩展能力

**节点扩展**:
- 动态hostfile: 支持任意数量节点加入集群
- 自动发现: DeepSpeed自动检测可用GPU资源
- 配置适配: 批次大小和学习率自动调整

**算力扩展**:
- GPU异构支持: 不同型号GPU混合部署
- 内存自适应: 根据GPU内存自动调整batch size
- 网络带宽适配: 自动检测和优化通信模式

#### 5.2 垂直扩展能力

**模型规模扩展**:
- ZeRO-3支持: 可切换到ZeRO-3支持更大模型
- 模型并行: 支持张量并行处理超大模型
- 流水线并行: 可扩展支持流水线并行训练

**数据规模扩展**:
- 流式数据加载: 支持TB级数据集训练
- 动态采样: 训练过程中动态调整数据采样策略
- 增量训练: 支持新数据增量微调

#### 5.3 功能扩展架构

**算法扩展**:
- TRL框架兼容: 支持PPO、DPO等其他RL算法
- 自定义奖励: 可替换不同的奖励模型
- 多目标优化: 支持多个奖励函数组合

**业务扩展**:
- 领域适配: 配置驱动的领域切换
- 多语言支持: 数据处理流水线支持多语言
- 评估扩展: 可插拔的评估指标和方法

### 六、生产环境部署架构

#### 6.1 部署自动化

**部署流水线**:
- rsync同步: 增量文件同步到目标服务器
- 权限管理: 自动设置脚本执行权限
- 环境检查: 自动验证依赖和配置

#### 6.2 容错与恢复

**故障检测**:
- 节点健康检查: 定期检测节点可用性
- 训练状态监控: 实时监控训练进度和指标
- 异常处理: 自动处理常见训练异常

**恢复机制**:
- 检查点恢复: 自动从最近检查点恢复训练
- 节点替换: 支持故障节点的热替换
- 数据完整性: 确保训练数据和模型状态一致性


## 代码讲解

### grpo_prompt.py  数据转换脚本

```python
# ==================== GRPO金融数据生成工具 ====================
# 作用：将英文金融新闻转换为中文GRPO训练数据
# 功能：批量处理、并发生成、格式验证、分层采样

# ==================== 库导入说明 ====================
import os                    # 操作系统相关功能，如文件路径操作、环境变量等
import json                  # 处理JSON格式数据的标准库
import time                  # 时间相关功能，如延迟等待、时间戳等
import random                # 随机数生成和随机采样功能
import openai                # OpenAI API客户端，这里用于调用DeepSeek API
from tqdm import tqdm        # 进度条显示库，用于显示处理进度
import concurrent.futures    # 并发处理库，用于多线程并行处理
import threading             # 线程相关功能，用于线程锁和同步
import argparse              # 用于解析命令行参数的标准库

# ==================== 全局配置参数 ====================
# 这些参数控制整个数据生成过程的行为
API_KEY = "sk-311f5262868yu79hnpej2a0986c16"  # DeepSeek API的密钥，用于身份验证
INPUT_FILE = "filtered_financial_news_5k.jsonl"    # 输入文件路径：包含英文金融新闻的数据文件
OUTPUT_FILE = "grpo_prompts_dataset_5k.jsonl"      # 输出文件路径：生成的GRPO训练数据
SAMPLE_COUNT = 50          # 需要采样的记录数量：从输入数据中选择50条进行处理
MAX_WORKERS = 80           # 并行处理的最大线程数：同时运行80个线程来加速处理
REQUEST_INTERVAL = 1       # 请求间隔（秒）：避免API调用过于频繁触发限制
RANDOM_SEED = 57           # 随机种子：确保每次运行程序时随机采样的结果都相同

# ==================== 随机种子设置 ====================
# 设置随机种子确保实验的可重现性
random.seed(RANDOM_SEED)

# ==================== 线程安全机制 ====================
# 多线程环境下需要使用锁来保护共享资源
print_lock = threading.Lock()   # 用于保护打印输出，避免多个线程同时打印导致输出混乱
output_lock = threading.Lock()  # 用于保护文件写入，避免多个线程同时写入同一文件导致数据损坏

# ==================== API客户端初始化 ====================
# 初始化DeepSeek API客户端
client = openai.OpenAI(
    api_key=API_KEY,                        # 使用上面定义的API密钥
    base_url="https://api.deepseek.com"     # DeepSeek的API服务地址
)

def truncate_text(text, max_length=5000):
    """
    截断文本以满足API长度限制
    
    参数：
        text: 需要截断的文本字符串
        max_length: 最大允许长度，默认5000字符
    
    返回：
        截断后的文本字符串
    
    作用：
    1. 检查文本长度是否超过限制
    2. 如果超过则从开头截取指定长度
    3. 避免API调用因文本过长而失败
    
    为什么需要截断：
    - LLM API通常对输入长度有限制
    - 过长的文本会导致API调用失败
    - 截断可以确保API调用的稳定性
    """
    if len(text) <= max_length:
        return text  # 文本长度合适，直接返回
    return text[:max_length]  # 截取前max_length个字符

def generate_grpo_prompt(item):
    """
    基于英文金融文章生成适用于GRPO训练的中文金融问题
    
    参数：
        item: 包含文章内容的字典，应该有'Article'字段
    
    返回：
        包含GRPO prompt的字典，格式：{"prompt": "问题内容"}
        如果失败返回None
    
    GRPO特点：
    1. 只需要生成问题（prompt），不需要答案
    2. 问题要足够详细，包含背景信息
    3. 支持模型生成多样化的回复用于强化学习
    
    生成策略：
    - 从英文文章中提取关键信息
    - 转换为详细的中文问题
    - 包含足够的背景信息支持回答
    - 设计开放性问题支持多样化回复
    """
    # 提取文章内容并截断
    article = item.get('Article', '')
    article_truncated = truncate_text(article)
    
    # 构建专门为GRPO训练优化的提示词
    # 这个提示词指导AI生成高质量的训练问题
    prompt = f"""
请基于以下英文金融文章，创建一个详细的中文金融问题，专门用于强化学习训练。

英文文章:
{article_truncated}

请完成以下任务:
1. 从文章中提取关键信息、数据、事实和核心观点
2. 创建一个专业的金融领域问题，要求该问题能够引导出多种不同的分析角度和回答方式
3. 问题必须包含足够详细的背景信息和事实，确保仅凭问题本身就能够推导出合理的回答
4. 问题应该具有一定的开放性，支持多样化的回答思路

输出格式必须是有效的JSON，结构如下:
{{
  "prompt": "这里是包含详细背景信息的专业金融问题"
}}

要求:
- 问题必须具体且深入，能够引导出金融专业领域的分析
- 必须包含足够丰富的事实信息，使回答者仅通过阅读问题就能回答
- 禁止出现"本文"、"文章"、"整体基调"、"情绪"等字样
- 禁止对文章本身进行评价或总结
- 直接以陈述事实的方式提供背景信息
- 问题应以客观的方式呈现数据和事实，避免主观评价
- 问题要以自然、符合实际提问习惯的方式表达
- 问题内容要特别详细，包含文章中所有能够支持回答问题的关键信息
- 问题应该具有一定的复杂性，支持多角度分析和思考
"""

    try:
        # 调用DeepSeek API生成GRPO问题
        response = client.chat.completions.create(
            model="deepseek-chat",           # 使用DeepSeek聊天模型
            messages=[
                {
                    "role": "system", 
                    "content": "你是一个专业的金融数据分析助手，精通英文金融文章翻译和问题构建。你的任务是创建适用于强化学习训练的专业金融问题，这些问题需要具有一定的开放性和复杂性。"
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            stream=False,                    # 不使用流式响应
            temperature=0.8                  # 提高创造性，生成更多样化的问题
        )
        
        # 提取生成的结果
        result = response.choices[0].message.content
        
        # 解析JSON响应
        # 寻找JSON开始和结束位置
        json_start = result.find('{')
        json_end = result.rfind('}') + 1
        
        if json_start != -1 and json_end != -1:
            # 提取JSON字符串并解析
            json_str = result[json_start:json_end]
            data = json.loads(json_str)
            
            # 验证JSON结构
            if "prompt" in data:
                return {"prompt": data["prompt"]}
            else:
                print("警告: 返回的JSON缺少'prompt'字段")
                return None
        else:
            print("无法在响应中找到有效的JSON")
            return None
            
    except Exception as e:
        print(f"生成GRPO prompt时出错: {e}")
        return None

def create_grpo_data(item, index, total):
    """
    生成GRPO训练数据：只生成prompt，不生成答案
    
    参数：
        item: 原始文章数据字典
        index: 当前处理的文章索引（从0开始）
        total: 总文章数量
    
    返回：
        GRPO训练数据字典，格式：{"prompt": "问题内容"}
        如果失败返回None
    
    GRPO数据特点：
    - 只包含prompt字段，不需要预定义的答案
    - 训练时模型会为每个prompt生成多个回复
    - 使用奖励模型对生成的回复进行评分
    - 通过强化学习优化模型参数
    
    处理流程：
    1. 记录处理进度
    2. 生成GRPO格式的问题
    3. 格式化为标准数据结构
    4. 处理生成失败的情况
    """
    # 使用线程锁确保打印输出不混乱
    with print_lock:
        print(f"处理文章 {index+1}/{total}")
    
    # 生成GRPO prompt
    prompt_data = generate_grpo_prompt(item)
    if not prompt_data:
        # 生成失败的处理
        with print_lock:
            print(f"文章 {index+1}/{total}: prompt生成失败")
        return None
    
    # 直接返回prompt数据，不需要其他字段
    # GRPO训练只需要prompt，不需要预定义答案
    grpo_data = {
        "prompt": prompt_data["prompt"]
    }
    
    # 记录成功信息
    with print_lock:
        print(f"文章 {index+1}/{total}: 成功生成GRPO prompt")
    
    return grpo_data

def process_article(args):
    """
    处理单篇文章的包装函数，专门用于并行处理
    
    参数：
        args: 包含处理参数的元组 (item, index, total, output_file)
            item: 文章数据
            index: 文章索引
            total: 总数量
            output_file: 输出文件路径
    
    返回：
        bool: 处理是否成功
    
    作用：
    1. 解包参数
    2. 添加随机延迟避免API限流
    3. 生成GRPO数据
    4. 安全地写入文件
    5. 返回处理结果
    
    为什么需要随机延迟：
    - API服务通常有频率限制
    - 同时大量请求可能被限流
    - 随机延迟可以分散请求时间
    """
    # 解包参数
    item, index, total, output_file = args
    
    # 添加随机延迟，避免同时大量请求API
    time.sleep(random.uniform(0, REQUEST_INTERVAL))
    
    # 生成GRPO数据
    result = create_grpo_data(item, index, total)
    if result:
        # 使用文件锁安全地写入结果
        with output_lock:
            with open(output_file, 'a', encoding='utf-8') as out_f:
                # 以JSONL格式写入（每行一个JSON对象）
                out_f.write(json.dumps(result, ensure_ascii=False) + '\n')
        return True
    return False

def stratified_random_sample(data_list, sample_count):
    """
    分层随机采样函数
    
    参数：
        data_list: 原始数据列表
        sample_count: 需要采样的数量
    
    返回：
        采样后的数据列表
    
    作用：
    1. 将数据分成多个层级
    2. 从每个层级中随机采样
    3. 确保采样的代表性
    4. 避免数据偏差
    
    分层采样的优势：
    - 比纯随机采样更有代表性
    - 确保样本覆盖整个数据集
    - 减少采样偏差
    - 提高样本质量
    """
    total_count = len(data_list)
    
    # 如果数据总量小于等于需要采样的数量，返回全部数据
    if total_count <= sample_count:
        print(f"数据总量({total_count})小于等于需要采样的数量({sample_count})，返回全部数据")
        return data_list
    
    # 确定分层数量
    # 至少10个数据为一层，但不超过采样数量
    num_strata = min(sample_count, total_count // 10 + 1)
    stratum_size = total_count // num_strata  # 每层的大小
    
    sampled_items = []
    
    # 从每个层级中采样
    for i in range(num_strata):
        # 计算当前层级的数据范围
        start_idx = i * stratum_size
        end_idx = start_idx + stratum_size if i < num_strata - 1 else total_count
        
        # 提取当前层级的数据
        stratum_data = data_list[start_idx:end_idx]
        
        # 计算当前层级应该采样的数量
        stratum_sample_count = max(1, int((end_idx - start_idx) / total_count * sample_count))
        
        # 确保不超过总采样数量
        if len(sampled_items) + stratum_sample_count > sample_count:
            stratum_sample_count = sample_count - len(sampled_items)
        
        # 从当前层级中采样
        if len(stratum_data) <= stratum_sample_count:
            # 如果层级数据不足，全部采用
            sampled_items.extend(stratum_data)
        else:
            # 随机采样指定数量
            stratum_samples = random.sample(stratum_data, stratum_sample_count)
            sampled_items.extend(stratum_samples)
        
        # 如果已达到采样数量，停止采样
        if len(sampled_items) >= sample_count:
            break

    # 如果采样数量仍不足，从剩余数据中补充
    if len(sampled_items) < sample_count:
        # 找出未被采样的数据
        remaining = [item for item in data_list if item not in sampled_items]
        # 从剩余数据中随机采样补足
        additional = random.sample(remaining, sample_count - len(sampled_items))
        sampled_items.extend(additional)
    
    return sampled_items

def parse_arguments():
    """
    解析命令行参数
    
    返回：
        argparse.Namespace: 解析后的参数对象
    
    作用：
    1. 定义所有可配置的参数
    2. 设置默认值
    3. 提供参数说明
    4. 支持命令行自定义配置
    
    支持的参数类别：
    - 数据相关：输入文件、输出文件、样本数量
    - API相关：API密钥
    - 并发相关：线程数、请求间隔
    - 其他：随机种子
    """
    parser = argparse.ArgumentParser(description="GRPO金融数据生成工具")
    
    # 数据相关参数
    parser.add_argument('--input_file', type=str, 
                       default='filtered_financial_news_5k.jsonl',
                       help='输入文件路径 (默认: filtered_financial_news_5k.jsonl)')
    
    parser.add_argument('--output_file', type=str,
                       default='grpo_prompts_dataset_5k.jsonl', 
                       help='输出文件路径 (默认: grpo_prompts_dataset_5k.jsonl)')
    
    parser.add_argument('--sample_count', type=int,
                       default=50,
                       help='需要处理的样本数量 (默认: 50)')
    
    # API相关参数
    parser.add_argument('--api_key', type=str,
                       default="sk-311f5262868e4c02b9a98cccb5986c16",
                       help='DeepSeek API密钥')
    
    # 并发相关参数
    parser.add_argument('--max_workers', type=int,
                       default=80,
                       help='最大并发线程数 (默认: 80)')
    
    parser.add_argument('--request_interval', type=float,
                       default=1.0,
                       help='请求间隔秒数 (默认: 1.0)')
    
    # 其他参数
    parser.add_argument('--random_seed', type=int,
                       default=57,
                       help='随机种子 (默认: 57)')
    
    return parser.parse_args()

def main():
    """
    主函数：协调整个GRPO数据处理流程
    
    作用：
    1. 解析命令行参数
    2. 更新全局配置
    3. 验证输入文件
    4. 读取和处理数据
    5. 执行并行处理
    6. 验证输出结果
    
    处理流程：
    1. 参数解析和配置更新
    2. 文件验证和数据加载
    3. 数据采样和格式化
    4. 并行生成GRPO数据
    5. 结果验证和示例展示
    """
    # 解析命令行参数
    args = parse_arguments()
    
    # 使用参数更新全局配置
    global INPUT_FILE, OUTPUT_FILE, SAMPLE_COUNT, API_KEY, MAX_WORKERS, REQUEST_INTERVAL, RANDOM_SEED
    INPUT_FILE = args.input_file
    OUTPUT_FILE = args.output_file
    SAMPLE_COUNT = args.sample_count
    API_KEY = args.api_key
    MAX_WORKERS = args.max_workers
    REQUEST_INTERVAL = args.request_interval
    RANDOM_SEED = args.random_seed
    
    # 重新设置随机种子确保一致性
    random.seed(RANDOM_SEED)
    
    # 重新初始化API客户端使用新的API密钥
    global client
    client = openai.OpenAI(
        api_key=API_KEY,
        base_url="https://api.deepseek.com"
    )
    
    # 打印配置信息
    print(f"=== GRPO数据生成配置 ===")
    print(f"输入文件: {INPUT_FILE}")
    print(f"输出文件: {OUTPUT_FILE}")
    print(f"样本数量: {SAMPLE_COUNT}")
    print(f"最大线程数: {MAX_WORKERS}")
    print(f"请求间隔: {REQUEST_INTERVAL}秒")
    print(f"随机种子: {RANDOM_SEED}")
    print(f"========================")
    
    # 确保输出目录存在
    os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)
    
    # 检查输入文件是否存在
    if not os.path.exists(INPUT_FILE):
        print(f"错误: 输入文件 '{INPUT_FILE}' 不存在!")
        return
    
    # 验证输入文件大小
    file_size = os.path.getsize(INPUT_FILE)
    print(f"输入文件大小: {file_size} 字节")
    if file_size == 0:
        print("错误: 输入文件为空!")
        return
    
    print(f"使用随机种子: {RANDOM_SEED} 确保可复现性")
    print(f"正在为GRPO训练生成prompt数据...")
    
    # ==================== 读取和解析输入文件 ====================
    items = []          # 存储有效数据
    line_count = 0      # 总行数
    error_count = 0     # 错误行数
    valid_count = 0     # 有效行数
    
    try:
        with open(INPUT_FILE, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                line_count += 1
                line = line.strip()  # 去除首尾空白
                if not line:         # 跳过空行
                    continue
                
                try:
                    # 解析JSON数据
                    item = json.loads(line)
                    
                    # 检查必需字段
                    if 'Article' in item:
                        items.append(item)
                        valid_count += 1
                    elif 'article' in item:
                        # 兼容小写字段名
                        item['Article'] = item['article']
                        items.append(item)
                        valid_count += 1
                    else:
                        print(f"警告: 第 {line_num} 行没有'Article'或'article'字段")
                        
                except json.JSONDecodeError as e:
                    # JSON解析错误
                    error_count += 1
                    print(f"错误: 第 {line_num} 行JSON解析失败: {e}")
                    
    except Exception as e:
        print(f"读取文件时发生错误: {e}")
    
    # 打印数据加载统计
    print(f"文件共有 {line_count} 行")
    print(f"解析错误: {error_count} 行")
    print(f"成功读取: {valid_count} 条有效记录")
    print(f"最终收集: {len(items)} 条记录")
    
    # 检查是否有有效数据
    if len(items) == 0:
        print("没有读取到有效数据!")
        return
    
    # ==================== 数据采样 ====================
    sampled_items = stratified_random_sample(items, SAMPLE_COUNT)
    print(f"分层随机采样了 {len(sampled_items)}/{len(items)} 条记录")
    
    # ==================== 创建输出文件 ====================
    # 清空输出文件（如果存在）
    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        pass
    
    # ==================== 并行处理 ====================
    print(f"开始并行处理，最大线程数: {MAX_WORKERS}")
    print(f"生成GRPO prompt数据...")
    
    # 准备并行处理的参数列表
    args_list = [(item, i, len(sampled_items), OUTPUT_FILE) for i, item in enumerate(sampled_items)]
    
    success_count = 0
    # 使用线程池执行并行处理
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # 执行所有任务并收集结果
        results = list(executor.map(process_article, args_list))
        # 统计成功数量
        success_count = sum(1 for r in results if r)
    
    # ==================== 处理结果统计 ====================
    print(f"GRPO数据生成完成，成功处理 {success_count}/{len(sampled_items)} 条记录")
    print(f"GRPO prompt数据已保存至 {OUTPUT_FILE}")
    
    # ==================== 验证输出文件 ====================
    if os.path.exists(OUTPUT_FILE):
        with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
            # 读取前3行作为示例
            sample_lines = [f.readline().strip() for _ in range(3)]
        
        print("\n生成的GRPO数据示例:")
        for i, line in enumerate(sample_lines, 1):
            if line:
                try:
                    data = json.loads(line)
                    # 只显示前100个字符避免输出过长
                    print(f"示例 {i}: {data['prompt'][:100]}...")
                except:
                    print(f"示例 {i}: 格式错误")

# ==================== 程序入口点 ====================
if __name__ == "__main__":
    main()  # 执行主函数
```

执行命令 
```python
python grpo_prompt.py \
    --input_file "filtered_financial_news_5k.jsonl" \
    --output_file "./test_prompts_dataset.jsonl" \
    --sample_count 10 \
    --max_workers 10 \
    --request_interval 2 \
    --random_seed 32
```

config_loader.py   配置加载器

```python 
"""
配置文件加载工具类
作用：为GRPO训练提供统一的配置管理功能
功能：支持JSON配置文件加载、嵌套配置访问、DeepSpeed配置管理
"""
import json                              # JSON数据处理
from pathlib import Path                 # 路径操作
from typing import Dict, Any, Optional   # 类型提示


class ConfigLoader:
    """
    配置文件加载器
    
    作用：负责加载和缓存JSON配置文件
    功能：
    1. 加载JSON配置文件
    2. 缓存配置以提高性能
    3. 保存配置到文件
    4. 处理文件读写错误
    """
    
    def __init__(self, config_dir: str = "configs"):
        """
        初始化配置加载器
        
        参数：
            config_dir: 配置文件目录路径
        """
        self.config_dir = Path(config_dir)  # 配置文件目录
        self._config_cache = {}             # 配置缓存字典
    
    def load_json(self, file_name: str) -> Dict[str, Any]:
        """
        加载JSON配置文件
        
        参数：
            file_name: 配置文件名
            
        返回：
            Dict: 配置字典
        
        作用：
        1. 检查缓存中是否已有配置
        2. 读取并解析JSON文件
        3. 缓存配置以提高后续访问速度
        4. 处理文件不存在和JSON格式错误
        """
        # 检查缓存
        if file_name in self._config_cache:
            return self._config_cache[file_name]
        
        # 构建文件路径
        file_path = self.config_dir / file_name
        
        # 检查文件是否存在
        if not file_path.exists():
            raise FileNotFoundError(f"配置文件不存在: {file_path}")
        
        try:
            # 读取并解析JSON文件
            with open(file_path, 'r', encoding='utf-8') as f:
                config = json.load(f)
            
            # 缓存配置
            self._config_cache[file_name] = config
            return config
            
        except json.JSONDecodeError as e:
            # JSON格式错误
            raise ValueError(f"配置文件JSON格式错误 {file_path}: {e}")
    
    def save_json(self, file_name: str, config: Dict[str, Any]):
        """
        保存配置到JSON文件
        
        参数：
            file_name: 配置文件名
            config: 配置字典
        
        作用：
        1. 确保目录存在
        2. 保存配置到JSON文件
        3. 更新缓存
        """
        file_path = self.config_dir / file_name
        
        # 确保目录存在
        file_path.parent.mkdir(parents=True, exist_ok=True)
        
        # 保存到文件
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(config, f, indent=2, ensure_ascii=False)
        
        # 更新缓存
        self._config_cache[file_name] = config


class ProjectConfig:
    """
    项目配置管理器
    
    作用：管理GRPO训练项目的所有配置
    功能：
    1. 支持嵌套配置访问（如 "model.model_name"）
    2. 提供配置的获取、设置、批量更新
    3. 配置文件保存和加载
    4. 配置信息打印
    """
    
    def __init__(self, config_file: str = "config.json"):
        """
        初始化项目配置
        
        参数：
            config_file: 配置文件名
        """
        self.config_loader = ConfigLoader()                    # 配置加载器
        self.config_file = config_file                         # 配置文件名
        self._config = self.config_loader.load_json(config_file)  # 加载配置
    
    def get(self, key_path: str, default: Any = None) -> Any:
        """
        获取配置值（支持嵌套键路径）
        
        参数：
            key_path: 键路径，如 "model.model_name"
            default: 默认值，当键不存在时返回
            
        返回：
            配置值
        
        作用：
        1. 支持点号分隔的嵌套路径访问
        2. 安全地处理不存在的键
        3. 提供默认值机制
        
        示例：
            config.get("model.model_name") -> "DeepSeek-R1-Distill-Qwen-7B"
            config.get("training.learning_rate") -> 5e-6
        """
        keys = key_path.split('.')  # 按点号分割路径
        value = self._config
        
        try:
            # 逐级访问嵌套字典
            for key in keys:
                value = value[key]
            return value
        except (KeyError, TypeError):
            # 键不存在或类型错误时返回默认值
            return default
    
    def set(self, key_path: str, value: Any):
        """
        设置配置值
        
        参数：
            key_path: 键路径
            value: 要设置的值
        
        作用：
        1. 支持嵌套路径设置
        2. 自动创建中间层级的字典
        3. 更新内存中的配置
        """
        keys = key_path.split('.')
        config = self._config
        
        # 导航到父级，创建中间层级
        for key in keys[:-1]:
            if key not in config:
                config[key] = {}
            config = config[key]
        
        # 设置最终值
        config[keys[-1]] = value
    
    def update(self, updates: Dict[str, Any]):
        """
        批量更新配置
        
        参数：
            updates: 更新字典，键为路径格式
        
        作用：一次性更新多个配置项
        
        示例：
            config.update({
                "training.learning_rate": 1e-5,
                "training.batch_size": 4
            })
        """
        for key_path, value in updates.items():
            self.set(key_path, value)
    
    def save(self):
        """
        保存配置到文件
        
        作用：将内存中的配置写入到原始配置文件
        """
        self.config_loader.save_json(self.config_file, self._config)
    
    def to_dict(self) -> Dict[str, Any]:
        """
        返回完整配置字典
        
        返回：
            Dict: 配置字典的副本
        
        作用：获取完整配置的副本，避免外部修改
        """
        return self._config.copy()
    
    def print_config(self):
        """
        打印当前配置
        
        作用：
        1. 格式化显示关键配置信息
        2. 便于调试和确认配置
        3. 在训练开始前进行配置检查
        """
        print("=" * 50)
        print("GRPO训练配置")
        print("=" * 50)
        print(f"基座模型: {self.get('model.model_name')}")
        print(f"奖励模型: {self.get('model.reward_model_path')}")
        
        # 检查是否使用LoRA
        lora_path = self.get('model.lora_model_path')
        if lora_path:
            print(f"LoRA模型: {lora_path}")
        
        # 数据和输出配置
        print(f"训练数据: {self.get('data.train_data_path')}")
        print(f"输出目录: {self.get('output.output_dir')}")
        
        # 训练超参数
        print(f"学习率: {self.get('training.learning_rate')}")
        print(f"训练轮数: {self.get('training.num_train_epochs')}")
        print(f"每GPU批次: {self.get('training.per_device_batch_size')}")
        print(f"梯度累积: {self.get('training.gradient_accumulation_steps')}")
        
        # GRPO特有参数
        print(f"生成样本数: {self.get('grpo.num_generations')}")
        print(f"使用Liger优化: {self.get('grpo.use_liger_loss')}")
        print("=" * 50)


class DeepSpeedConfig:
    """
    DeepSpeed配置管理器
    
    作用：管理DeepSpeed分布式训练的配置
    功能：
    1. 加载DeepSpeed配置文件
    2. 分离Accelerate和DeepSpeed配置
    3. 动态更新GPU数量配置
    4. 支持配置导出为YAML格式
    
    DeepSpeed是什么：
    - 微软开发的分布式训练框架
    - 支持ZeRO优化器状态分片
    - 可以训练超大规模模型
    - 与PyTorch和Transformers集成
    """
    
    def __init__(self, config_file: str = "deepspeed_zero3.json"):
        """
        初始化DeepSpeed配置
        
        参数：
            config_file: DeepSpeed配置文件名
        """
        self.config_loader = ConfigLoader()
        self.config_file = config_file
        self._config = self.config_loader.load_json(config_file)
    
    def get_accelerate_config(self) -> Dict[str, Any]:
        """
        获取Accelerate使用的配置部分
        
        返回：
            Accelerate配置字典
        
        作用：
        1. 过滤掉DeepSpeed特有的配置
        2. 返回Accelerate框架需要的配置
        3. 支持Accelerate + DeepSpeed集成
        """
        accelerate_config = {}
        exclude_keys = ['deepspeed_config']  # 排除的键
        
        for key, value in self._config.items():
            if key not in exclude_keys:
                accelerate_config[key] = value
        
        return accelerate_config
    
    def get_deepspeed_config(self) -> Dict[str, Any]:
        """
        获取DeepSpeed引擎配置
        
        返回：
            DeepSpeed配置字典
        
        作用：提取DeepSpeed引擎需要的配置部分
        """
        return self._config.get('deepspeed_config', {})
    
    def update_gpu_count(self, num_gpus: int):
        """
        更新GPU数量配置
        
        参数：
            num_gpus: GPU数量
        
        作用：
        1. 更新进程数量配置
        2. 重新计算全局batch size
        3. 确保配置与硬件匹配
        
        """
        # 更新进程数量
        self._config['num_processes'] = num_gpus
        
        # 获取DeepSpeed配置
        deepspeed_config = self._config.get('deepspeed_config', {})
        micro_batch_size = deepspeed_config.get('train_micro_batch_size_per_gpu', 1)
        gradient_accumulation = deepspeed_config.get('gradient_accumulation_steps', 4)
        
        # 计算全局batch size
        global_batch_size = num_gpus * micro_batch_size * gradient_accumulation
        deepspeed_config['train_batch_size'] = global_batch_size
    
    def save_to_yaml(self, output_path: str):
        """
        保存为YAML格式（兼容accelerate launch）
        
        参数：
            output_path: 输出文件路径
        
        作用：
        1. 转换为YAML格式
        2. 兼容accelerate launch命令
        3. 便于人类阅读和编辑
        """
        try:
            import yaml
        except ImportError:
            raise ImportError("需要安装PyYAML: pip install PyYAML")
        
        with open(output_path, 'w', encoding='utf-8') as f:
            yaml.dump(self._config, f, default_flow_style=False, allow_unicode=True)
    
    def to_dict(self) -> Dict[str, Any]:
        """
        返回完整配置字典
        
        返回：
            Dict: 配置字典的副本
        """
        return self._config.copy()


# 全局配置实例创建函数
def load_configs():
    """
    加载所有配置
    
    返回：
        Tuple: (项目配置, DeepSpeed配置)
    
    作用：一次性加载项目需要的所有配置
    """
    project_config = ProjectConfig()
    deepspeed_config = DeepSpeedConfig()
    return project_config, deepspeed_config


# 便捷函数
def get_project_config() -> ProjectConfig:
    """
    获取项目配置实例
    
    返回：
        ProjectConfig: 项目配置实例
    
    作用：提供全局访问项目配置的便捷方法
    """
    return ProjectConfig()


def get_deepspeed_config() -> DeepSpeedConfig:
    """
    获取DeepSpeed配置实例
    
    返回：
        DeepSpeedConfig: DeepSpeed配置实例
    
    作用：提供全局访问DeepSpeed配置的便捷方法
    """
    return DeepSpeedConfig()

```

logger.py 日志管理

```python
"""
日志工具类
作用：为GRPO训练过程提供统一的日志记录功能
功能：支持控制台输出、文件记录、训练过程跟踪
"""
import logging                    # Python标准日志库
import sys                       # 系统相关功能
from datetime import datetime    # 日期时间处理
from pathlib import Path        # 路径操作工具


class Logger:
    """
    训练日志管理器
    
    作用：统一管理GRPO训练过程中的所有日志输出
    功能：
    1. 支持多种日志级别（INFO、WARNING、ERROR、DEBUG）
    2. 同时输出到控制台和文件
    3. 提供训练专用的日志记录方法
    4. 自动格式化日志消息
    """
    
    def __init__(self, name: str = "GRPO", log_level: str = "INFO", 
                 log_file: str = None, console_output: bool = True):
        """
        初始化日志器
        
        参数说明：
            name: 日志器名称，用于区分不同模块的日志
            log_level: 日志级别（DEBUG < INFO < WARNING < ERROR）
            log_file: 日志文件路径，如果为None则不保存到文件
            console_output: 是否同时输出到控制台
        """
        # 创建Python标准日志器实例
        self.logger = logging.getLogger(name)
        # 设置日志级别 - 只有达到此级别的消息才会被记录
        self.logger.setLevel(getattr(logging, log_level.upper()))
        
        # 清除已有的处理器，避免重复输出
        self.logger.handlers.clear()
        
        # 设置日志消息格式
        # 格式：时间 - 日志器名称 - 级别 - 消息内容
        formatter = logging.Formatter(
            '%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S'  # 时间格式
        )
        
        # 控制台输出处理器
        if console_output:
            console_handler = logging.StreamHandler(sys.stdout)  # 输出到标准输出
            console_handler.setFormatter(formatter)              # 应用格式
            self.logger.addHandler(console_handler)              # 添加到日志器
        
        # 文件输出处理器
        if log_file:
            # 确保日志目录存在
            Path(log_file).parent.mkdir(parents=True, exist_ok=True)
            file_handler = logging.FileHandler(log_file, encoding='utf-8')  # 创建文件处理器
            file_handler.setFormatter(formatter)                           # 应用格式
            self.logger.addHandler(file_handler)                           # 添加到日志器
    
    def info(self, message: str):
        """
        输出信息日志
        用途：记录一般性信息，如训练进度、状态更新等
        """
        self.logger.info(message)
    
    def warning(self, message: str):
        """
        输出警告日志
        用途：记录可能的问题，但不影响程序继续运行
        """
        self.logger.warning(message)
    
    def error(self, message: str):
        """
        输出错误日志
        用途：记录严重错误，可能导致程序异常
        """
        self.logger.error(message)
    
    def debug(self, message: str):
        """
        输出调试日志
        用途：记录详细的调试信息，通常只在开发阶段使用
        """
        self.logger.debug(message)
    
    def log_training_start(self, config):
        """
        记录训练开始信息
        
        参数：
            config: 训练配置对象，包含所有训练参数
        
        作用：在训练开始时记录完整的配置信息，便于后续追踪
        """
        self.info("=" * 60)
        self.info("GRPO金融模型训练开始")
        self.info("=" * 60)
        self.info(f"开始时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        # 记录关键配置信息
        # 使用嵌套路径访问配置项（如 'model.model_name'）
        self.info(f"基座模型: {config.get('model.model_name')}")
        self.info(f"奖励模型: {config.get('model.reward_model_path')}")
        
        # 检查是否使用LoRA微调
        lora_path = config.get('model.lora_model_path')
        if lora_path:
            self.info(f"LoRA模型: {lora_path}")
        
        # 记录数据和输出配置
        self.info(f"训练数据: {config.get('data.train_data_path')}")
        self.info(f"输出目录: {config.get('output.output_dir')}")
        
        # 记录训练超参数
        self.info(f"学习率: {config.get('training.learning_rate')}")
        self.info(f"训练轮数: {config.get('training.num_train_epochs')}")
        self.info(f"批次大小: {config.get('training.per_device_batch_size')}")
        self.info(f"梯度累积: {config.get('training.gradient_accumulation_steps')}")
        self.info("=" * 60)
    
    def log_training_end(self, success: bool = True):
        """
        记录训练结束信息
        
        参数：
            success: 训练是否成功完成
        
        作用：记录训练结束状态和时间
        """
        self.info("=" * 60)
        if success:
            self.info("GRPO训练成功完成！")
        else:
            self.error("GRPO训练失败！")
        self.info(f"结束时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        self.info("=" * 60)
    
    def log_step(self, step: int, loss: float, learning_rate: float, 
                 reward_mean: float = None):
        """
        记录训练步骤信息
        
        参数：
            step: 当前训练步数
            loss: 当前损失值
            learning_rate: 当前学习率
            reward_mean: 平均奖励值（GRPO特有）
        
        作用：记录每个训练步骤的关键指标
        """
        msg = f"Step {step} - Loss: {loss:.4f} - LR: {learning_rate:.2e}"
        if reward_mean is not None:
            msg += f" - Reward: {reward_mean:.4f}"
        self.info(msg)


# 创建全局日志实例的工厂函数
def create_logger(output_dir: str = "./output", log_level: str = "INFO") -> Logger:
    """
    创建日志实例
    
    参数：
        output_dir: 输出目录，日志文件将保存在此目录下
        log_level: 日志级别
    
    返回：
        Logger实例
    
    作用：为整个项目提供统一的日志创建入口
    """
    # 创建日志文件路径
    log_file = Path(output_dir) / "training.log"
    
    return Logger(
        name="GRPO_Training",      # 日志器名称
        log_level=log_level,       # 日志级别
        log_file=str(log_file),    # 日志文件路径
        console_output=True        # 启用控制台输出
    )
```

### dataset.py    数据集处理

```python
"""
数据集处理工具类
作用：为GRPO训练提供数据加载和预处理功能
功能：支持JSONL格式数据加载、对话格式化、奖励函数创建
"""
import json                              # JSON数据处理
from typing import List, Dict, Any       # 类型提示
from pathlib import Path                 # 路径操作
from datasets import Dataset             # Hugging Face数据集库
from transformers import PreTrainedTokenizer  # 分词器基类
import torch                            # PyTorch深度学习框架


class GRPODatasetLoader:
    """
    GRPO数据集加载器
    
    作用：专门为GRPO训练加载和处理数据
    功能：
    1. 加载JSONL格式的训练数据
    2. 将数据格式化为对话格式
    3. 创建适合GRPO训练的数据集
    4. 处理数据加载过程中的错误
    """
    
    def __init__(self, tokenizer: PreTrainedTokenizer, logger=None):
        """
        初始化数据集加载器
        
        参数：
            tokenizer: 分词器实例，用于文本处理
            logger: 日志器实例，用于记录处理过程
        """
        self.tokenizer = tokenizer  # 保存分词器引用
        self.logger = logger        # 保存日志器引用
    
    def load_jsonl(self, file_path: str) -> List[Dict[str, Any]]:
        """
        加载JSONL格式数据文件
        
        参数：
            file_path: 数据文件路径
            
        返回：
            List[Dict]: 解析后的数据列表，每个元素是一个字典
        
        作用：
        1. 逐行读取JSONL文件
        2. 解析每行的JSON数据
        3. 验证数据格式
        4. 统计加载结果
        """
        # 检查文件是否存在
        if not Path(file_path).exists():
            raise FileNotFoundError(f"数据文件不存在: {file_path}")
        
        data = []           # 存储有效数据
        error_count = 0     # 错误计数
        
        # 逐行读取文件
        with open(file_path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                line = line.strip()  # 去除首尾空白
                if not line:         # 跳过空行
                    continue
                
                try:
                    # 解析JSON数据
                    item = json.loads(line)
                    # 验证必需字段
                    if 'prompt' in item:
                        data.append(item)
                    else:
                        if self.logger:
                            self.logger.warning(f"第{line_num}行缺少'prompt'字段")
                        error_count += 1
                except json.JSONDecodeError as e:
                    # JSON解析错误
                    if self.logger:
                        self.logger.error(f"第{line_num}行JSON解析失败: {e}")
                    error_count += 1
        
        # 记录加载结果
        if self.logger:
            self.logger.info(f"数据加载完成: {len(data)}条有效记录, {error_count}条错误记录")
        
        return data
    
    def format_chat_prompt(self, prompt: str) -> List[Dict[str, str]]:
        """
        将prompt格式化为对话格式
        
        参数：
            prompt: 原始prompt文本
            
        返回：
            List[Dict]: 格式化后的对话消息列表
        
        作用：
        1. 将单一的prompt转换为多轮对话格式
        2. 添加系统提示词，定义AI助手的角色
        3. 将用户问题包装为用户消息
        
        GRPO训练需要对话格式的数据，因为：
        - 现代语言模型都是基于对话训练的
        - 对话格式可以更好地控制模型行为
        - 系统提示词可以定义专业领域角色
        """
        return [
            {
                "role": "system",  # 系统角色
                "content": "你是一个专业的金融分析师，擅长投资分析和风险评估。请基于提供的信息进行详细的分析和推理。"
            },
            {
                "role": "user",    # 用户角色
                "content": prompt  # 用户问题
            }
        ]
    
    def create_dataset(self, data_path: str) -> Dataset:
        """
        创建GRPO训练数据集
        
        参数：
            data_path: 数据文件路径
            
        返回：
            Dataset: HuggingFace数据集对象
        
        作用：
        1. 加载原始数据
        2. 格式化为对话格式
        3. 转换为HuggingFace数据集
        4. 保留原始数据的其他字段
        
        GRPO训练的数据特点：
        - 只需要prompt，不需要预定义答案
        - 训练时模型会生成多个回复
        - 使用奖励模型评分和优化
        """
        if self.logger:
            self.logger.info(f"开始加载数据集: {data_path}")
        
        # 加载原始数据
        raw_data = self.load_jsonl(data_path)
        
        # 格式化数据
        formatted_data = []
        for item in raw_data:
            # 格式化prompt为对话格式
            formatted_item = {
                "prompt": self.format_chat_prompt(item["prompt"])
            }
            # 保留其他字段（如果有的话）
            for key, value in item.items():
                if key != "prompt":
                    formatted_item[key] = value
            
            formatted_data.append(formatted_item)
        
        # 转换为HuggingFace数据集
        dataset = Dataset.from_list(formatted_data)
        
        if self.logger:
            self.logger.info(f"数据集创建完成: {len(dataset)}条记录")
        
        return dataset


def create_reward_function(reward_model_path: str, logger=None):
    """
    创建奖励函数
    
    参数：
        reward_model_path: 奖励模型路径
        logger: 日志器实例
    
    返回：
        reward_function: 奖励计算函数
    
    作用：
    1. 加载预训练的奖励模型
    2. 创建GRPO兼容的奖励函数
    3. 确保文本格式与奖励模型训练时一致
    
    奖励函数的重要性：
    - GRPO是强化学习算法，需要奖励信号指导训练
    - 奖励模型评估生成文本的质量
    - 文本格式必须与奖励模型训练时完全一致
    """
    from transformers import AutoTokenizer, AutoModelForSequenceClassification
    
    if logger:
        logger.info(f"加载奖励模型: {reward_model_path}")
    
    # 为奖励模型使用专用的分词器
    # 重要：奖励模型可能使用不同的分词器配置
    reward_tokenizer = AutoTokenizer.from_pretrained(
        reward_model_path,
        trust_remote_code=True
    )
    
    # 设置填充token
    if reward_tokenizer.pad_token is None:
        reward_tokenizer.pad_token = reward_tokenizer.eos_token
    
    # 加载奖励模型
    # 奖励模型是一个分类模型，输出单一分数
    reward_model = AutoModelForSequenceClassification.from_pretrained(
        reward_model_path,
        num_labels=1,                    # 输出单一奖励分数
        torch_dtype=torch.bfloat16,      # 使用半精度节省显存
        trust_remote_code=True,
        use_cache=False                  # 禁用缓存节省内存
    )
    reward_model.eval()  # 设置为评估模式
    
    if logger:
        logger.info("奖励模型加载完成")
    
    def reward_function(prompts, completions, **kwargs):
        """
        GRPO兼容的奖励函数 - 使用与奖励模型训练一致的LLaMA-3格式
        
        参数：
            prompts: 输入提示列表
            completions: 模型生成的完成文本列表
            **kwargs: 其他参数
        
        返回：
            rewards: 奖励分数列表
        
        关键修复：
        - 使用与奖励模型训练完全相同的文本格式
        - LLaMA-3格式的特殊token必须一致
        - 避免格式不匹配导致的奖励异常
        """
        rewards = []
        
        # 禁用梯度计算，节省内存
        with torch.no_grad():
            for prompt, completion in zip(prompts, completions):
                try:
                    # 提取用户问题
                    if isinstance(prompt, list):
                        # 从对话格式中提取用户问题
                        question = next(msg["content"] for msg in prompt if msg["role"] == "user")
                    else:
                        question = prompt
                    
                    # 处理completion
                    if isinstance(completion, list):
                        completion_text = completion[0]["content"]
                    else:
                        completion_text = completion
                    
                    # 关键修复：使用与奖励模型训练完全相同的LLaMA-3格式
                    # 这个格式必须与奖励模型训练时的格式完全一致
                    formatted_text = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{completion_text}<|eot_id|>"
                    
                    # 使用奖励模型计算分数
                    inputs = reward_tokenizer(
                        formatted_text,
                        truncation=True,           # 截断过长文本
                        padding=True,              # 填充到统一长度
                        max_length=2048,          # 与奖励模型训练时一致
                        return_tensors="pt"        # 返回PyTorch张量
                    )
                    
                    # 将输入移动到模型设备
                    inputs = {k: v.to(reward_model.device) for k, v in inputs.items()}
                    
                    # 模型推理
                    outputs = reward_model(**inputs)
                    
                    # 提取奖励分数
                    reward = outputs.logits.squeeze().cpu().item()
                    rewards.append(reward)
                    
                except Exception as e:
                    # 处理计算失败的情况
                    if logger:
                        logger.warning(f"奖励计算失败: {e}")
                    rewards.append(0.0)  # 使用默认分数
        
        return rewards
    
    return reward_function
```

### train.py  主训练脚本

```python
"""
GRPO训练主文件 - 简单修复DeepSpeed参数传递
作用：实现完整的GRPO强化学习训练流程
功能：包含模型加载、数据处理、训练器配置、最佳模型保存等
"""
import torch

# ==================== PyTorch加载修复 ====================
# 修复checkpoint加载时的weights_only问题
# 背景：新版PyTorch对checkpoint加载有更严格的安全检查
original_torch_load = torch.load  # 保存原始加载函数

def patched_torch_load(f, map_location=None, pickle_module=None, weights_only=None, **kwargs):
    """
    修复的torch.load函数
    
    参数：
        f: 文件路径或文件对象
        map_location: 设备映射
        pickle_module: pickle模块
        weights_only: 是否只加载权重
        **kwargs: 其他参数
    
    作用：
    1. 检测是否是checkpoint相关文件
    2. 对于checkpoint文件，强制设置weights_only=False
    3. 避免PyTorch安全检查导致的加载失败
    """
    # 如果是加载checkpoint相关文件，强制设置weights_only=False
    # 这样可以避免某些版本的PyTorch过于严格的安全检查
    if weights_only is True and (
        isinstance(f, str) and ('rng_state' in f or 'checkpoint' in f)
    ):
        weights_only = False
    return original_torch_load(f, map_location=map_location, pickle_module=pickle_module, 
                              weights_only=weights_only, **kwargs)

# 替换PyTorch的加载函数
torch.load = patched_torch_load

# ==================== 导入必要的库 ====================
import time                                      # 时间处理
from pathlib import Path                         # 路径操作
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainerCallback  # Transformers库
from peft import PeftModel                       # LoRA等参数高效微调
from trl import GRPOConfig, GRPOTrainer         # TRL强化学习库
import argparse                                  # 命令行参数解析
import numpy as np                               # 数值计算
import os                                        # 操作系统接口
from utils.config_loader import ProjectConfig   # 项目配置加载器
from utils.logger import create_logger          # 日志工具
from utils.dataset import GRPODatasetLoader, create_reward_function  # 数据集和奖励函数


class GRPOBestModelEarlyStoppingCallback(TrainerCallback):
    """
    GRPO最佳模型早停回调
    
    作用：
    1. 自动保存最佳模型
    2. 实现早停机制，防止过拟合
    3. 监控训练和验证指标
    4. 管理模型保存策略
    
    GRPO训练的特点：
    - 强化学习训练可能不稳定
    - 需要持续监控奖励指标
    - 最佳模型可能出现在训练中期
    - 需要保存最佳性能的模型而非最后的模型
    """
    
    def __init__(
        self, 
        early_stopping_patience: int = 3,           # 早停耐心值
        early_stopping_threshold: float = 0.001,    # 早停阈值
        train_reward_weight: float = 0.3,           # 训练奖励权重
        eval_reward_weight: float = 0.7,            # 验证奖励权重
        min_eval_steps: int = 2,                    # 最小验证步数
        best_model_dir: str = None,                 # 最佳模型保存目录
        tokenizer = None                            # 分词器实例
    ):
        """
        初始化早停回调
        
        参数说明：
            early_stopping_patience: 连续多少次验证无改善后停止训练
            early_stopping_threshold: 认为有改善的最小阈值
            train_reward_weight: 训练奖励在综合评分中的权重
            eval_reward_weight: 验证奖励在综合评分中的权重
            min_eval_steps: 开始早停检查前的最小验证步数
            best_model_dir: 最佳模型保存目录
            tokenizer: 分词器实例，用于保存
        """
        # 早停参数
        self.early_stopping_patience = early_stopping_patience
        self.early_stopping_threshold = early_stopping_threshold
        self.train_reward_weight = train_reward_weight
        self.eval_reward_weight = eval_reward_weight
        self.min_eval_steps = min_eval_steps
        
        # 保存tokenizer引用
        self.tokenizer = tokenizer
        
        # 最佳模型保存路径
        self.best_model_dir = best_model_dir or "./output/best_model"
        Path(self.best_model_dir).mkdir(parents=True, exist_ok=True)
        
        # 训练指标跟踪
        self.latest_train_reward = 0.0      # 最新训练奖励
        self.best_train_reward = None       # 最佳训练奖励
        
        # 验证指标跟踪
        self.best_eval_reward = None        # 最佳验证奖励
        self.eval_count = 0                 # 验证次数计数
        
        # 早停控制变量
        self.patience_counter = 0           # 耐心计数器
        self.best_combined_score = None     # 最佳综合得分
        self.best_step = None               # 最佳模型对应的步数
        
        # 调试计数器
        self.log_call_count = 0             # 日志调用次数
        self.eval_call_count = 0            # 验证调用次数
        
        print(f"回调函数初始化: 训练权重={train_reward_weight}, 验证权重={eval_reward_weight}")
        print(f"最佳模型保存目录: {self.best_model_dir}")
        print(f"分词器引用: {self.tokenizer is not None}")
    
    def on_log(self, args, state, control, logs=None, **kwargs):
        """
        监听所有日志事件
        
        参数：
            args: 训练参数
            state: 训练状态
            control: 训练控制
            logs: 日志字典
            **kwargs: 其他参数（包含model等）
        
        作用：
        1. 监听训练过程中的所有日志
        2. 识别验证日志并触发模型保存
        3. 更新训练奖励记录
        4. 实现最佳模型保存逻辑
        """
        self.log_call_count += 1
        
        print(f"on_log调用#{self.log_call_count} - Step {state.global_step}")
        
        if logs:
            # 检查是否是验证日志
            # 验证日志的特征是包含以'eval_'开头的键
            if any(key.startswith('eval_') for key in logs.keys()):
                print("检测到验证日志，尝试保存最佳模型...")
                
                # 获取验证奖励
                eval_reward = logs.get('eval_reward')
                if eval_reward is not None:
                    print(f"验证奖励: {eval_reward}")
                    
                    # 判断是否需要保存模型
                    should_save = False
                    if self.best_eval_reward is None or eval_reward > self.best_eval_reward:
                        self.best_eval_reward = eval_reward
                        should_save = True
                        print(f"发现更好的验证奖励: {eval_reward}")
                    
                    # 第一次验证时强制保存
                    if self.eval_count == 0:
                        should_save = True
                        print("第一次验证，强制保存模型")
                    
                    if should_save:
                        # 从kwargs中获取模型实例
                        model = kwargs.get('model')
                        
                        if model and self.tokenizer:
                            self._save_best_model(model, self.tokenizer, state.global_step)
                        else:
                            print(f"无法获取model或tokenizer: model={model is not None}, tokenizer={self.tokenizer is not None}")
                    
                    self.eval_count += 1
            
            # 获取训练奖励
            # GRPO训练中奖励可能以不同的键名出现
            possible_reward_keys = ['reward', 'rewards/reward_function/mean', 'train_reward', 'reward_mean']
            for key in possible_reward_keys:
                if key in logs:
                    self.latest_train_reward = logs[key]
                    break
        
        return control
    
    def on_evaluate(self, args, state, control, logs=None, **kwargs):
        """
        验证回调 - 简化版本
        
        参数：
            args: 训练参数
            state: 训练状态
            control: 训练控制
            logs: 日志字典
            **kwargs: 其他参数
        
        作用：
        1. 记录验证回调的调用
        2. 验证逻辑已经移到on_log中处理
        
        注意：实际的验证处理逻辑在on_log方法中
        """
        self.eval_call_count += 1
        print(f"on_evaluate调用#{self.eval_call_count} - Step {state.global_step}")
        
        # 验证逻辑已经移到 on_log 中处理
        return control
    
    def _save_best_model(self, model, tokenizer, step):
        """
        保存最佳模型
        
        参数：
            model: 模型实例
            tokenizer: 分词器实例
            step: 当前训练步数
        
        作用：
        1. 处理DeepSpeed包装的模型
        2. 保存模型权重和配置
        3. 保存分词器
        4. 记录模型元信息
        5. 处理保存过程中的异常
        """
        try:
            print(f"开始保存最佳模型 (Step {step})...")
            
            # 确保目录存在
            Path(self.best_model_dir).mkdir(parents=True, exist_ok=True)
            
            # 处理DeepSpeed包装的模型
            # DeepSpeed会用module属性包装原始模型
            actual_model = model.module if hasattr(model, 'module') else model
                
            # 保存模型
            actual_model.save_pretrained(
                self.best_model_dir,              # 保存目录
                safe_serialization=True,          # 使用安全序列化
                max_shard_size="2GB"              # 最大分片大小
            )
            
            # 保存分词器
            tokenizer.save_pretrained(self.best_model_dir)
            
            # 保存元信息
            # 记录模型的关键指标和保存时间
            best_info = {
                "best_step": step,                         # 最佳模型对应的步数
                "best_combined_score": self.best_combined_score,  # 最佳综合得分
                "best_train_reward": self.best_train_reward,      # 最佳训练奖励
                "best_eval_reward": self.best_eval_reward,        # 最佳验证奖励
                "latest_train_reward": self.latest_train_reward,  # 最新训练奖励
                "save_time": time.strftime("%Y-%m-%d %H:%M:%S")   # 保存时间
            }
            
            import json
            with open(f"{self.best_model_dir}/best_model_info.json", "w") as f:
                json.dump(best_info, f, indent=2)
                
            print(f"最佳模型保存完成！文件数量: {len(list(Path(self.best_model_dir).glob('*')))}")
            
        except Exception as e:
            print(f"保存最佳模型失败: {e}")
            import traceback
            traceback.print_exc()  # 打印完整的错误堆栈


class GRPOTraining:
    """
    GRPO训练主类
    
    作用：
    1. 统一管理GRPO训练的所有组件
    2. 按照标准流程初始化各个模块
    3. 提供完整的训练接口
    4. 处理训练过程中的异常
    
    训练流程：
    1. 设置分词器
    2. 加载策略模型
    3. 准备训练数据
    4. 创建奖励函数
    5. 配置训练器
    6. 执行训练
    """
    
    def __init__(self, config: ProjectConfig = None, deepspeed_config: str = None):
        """
        初始化GRPO训练器
        
        参数：
            config: 项目配置实例
            deepspeed_config: DeepSpeed配置文件路径
        """
        self.config = config or ProjectConfig()          # 项目配置
        self.deepspeed_config = deepspeed_config         # DeepSpeed配置路径
        
        # 创建输出目录
        output_dir = self.config.get('output.output_dir')
        Path(output_dir).mkdir(parents=True, exist_ok=True)
        
        # 创建日志器
        self.logger = create_logger(output_dir, 'INFO')
        
        # 初始化组件引用
        self.tokenizer = None        # 分词器
        self.model = None           # 策略模型
        self.dataset = None         # 训练数据集
        self.reward_function = None # 奖励函数
        self.trainer = None         # GRPO训练器
    
    def setup_tokenizer(self):
        """
        设置分词器
        
        作用：
        1. 加载预训练分词器
        2. 配置特殊token
        3. 设置填充策略
        
        分词器的重要性：
        - 将文本转换为模型可理解的数字序列
        - 处理特殊token（如开始、结束、填充token）
        - 确保文本格式与模型训练时一致
        """
        self.logger.info("设置分词器...")
        model_name = self.config.get('model.model_name')
        
        # 加载分词器
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name,                    # 模型名称或路径
            trust_remote_code=True,        # 允许执行远程代码
            padding_side="left"            # 左侧填充（生成任务推荐）
        )
        
        # 设置填充token
        # 如果模型没有定义填充token，使用结束token代替
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        self.logger.info("分词器设置完成")
    
    def setup_model(self):
        """
        设置策略模型
        
        作用：
        1. 加载预训练的基座模型
        2. 可选地加载LoRA适配器
        3. 配置模型参数
        4. 确保与DeepSpeed兼容
        
        模型加载策略：
        - 使用bfloat16精度节省显存
        - 禁用缓存减少内存占用
        - 让DeepSpeed处理模型分布
        - 确保所有参数可训练
        """
        self.logger.info("设置策略模型...")
        model_name = self.config.get('model.model_name')
        
        # 让DeepSpeed处理模型分布
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,                    # 模型名称或路径
            torch_dtype=torch.bfloat16,    # 使用半精度浮点
            trust_remote_code=True,        # 允许执行远程代码
            use_cache=False,               # 禁用KV缓存节省内存
            device_map=None,               # 关键：让DeepSpeed处理设备分配
            low_cpu_mem_usage=True         # 低CPU内存使用模式
        )
        
        # 加载LoRA模型（如果配置了）
        # LoRA是一种参数高效的微调方法
        lora_path = self.config.get('model.lora_model_path')
        if lora_path:
            self.model = PeftModel.from_pretrained(self.model, lora_path)
        
        # 确保参数可训练
        # 某些情况下模型参数可能被冻结
        for param in self.model.parameters():
            param.requires_grad = True
        
        self.logger.info("策略模型设置完成")
    
    def setup_dataset(self):
        """
        设置数据集
        
        作用：
        1. 创建数据集加载器
        2. 加载训练数据
        3. 加载验证数据
        4. 格式化为GRPO训练格式
        
        GRPO数据特点：
        - 只需要prompt，不需要答案
        - 使用对话格式
        - 支持批量处理
        """
        self.logger.info("设置数据集...")
        dataset_loader = GRPODatasetLoader(self.tokenizer, self.logger)
        
        # 加载训练数据
        data_path = self.config.get('data.train_data_path')
        self.dataset = dataset_loader.create_dataset(data_path)
        
        # 加载验证数据
        eval_data_path = self.config.get('data.eval_data_path')
        self.eval_dataset = dataset_loader.create_dataset(eval_data_path)
        
        self.logger.info(f"数据集设置完成: {len(self.dataset)}条记录")
    
    def setup_reward_function(self):
        """
        设置奖励函数
        
        作用：
        1. 加载预训练的奖励模型
        2. 创建GRPO兼容的奖励函数
        3. 确保文本格式一致性
        
        奖励函数的作用：
        - 评估模型生成文本的质量
        - 为GRPO提供训练信号
        - 指导模型向更好的方向优化
        """
        self.logger.info("设置奖励函数...")
        reward_model_path = self.config.get('model.reward_model_path')
        self.reward_function = create_reward_function(reward_model_path, self.logger)
        self.logger.info("奖励函数设置完成")
    
    def setup_trainer(self):
        """
        设置GRPO训练器
        
        作用：
        1. 配置训练参数
        2. 创建早停回调
        3. 初始化GRPO训练器
        4. 集成所有组件
        
        训练器配置包括：
        - 基本训练参数（学习率、批次大小等）
        - GRPO特有参数（生成数量、温度等）
        - 保存和评估策略
        - 日志和监控设置
        """
        self.logger.info("设置GRPO训练器...")
        
        # 创建最佳模型早停回调
        best_model_dir = f"{self.config.get('output.output_dir')}best_model"
        early_stopping_callback = GRPOBestModelEarlyStoppingCallback(
            early_stopping_patience=self.config.get('training.early_stopping_patience', 3),
            early_stopping_threshold=self.config.get('training.early_stopping_threshold', 0.001),
            best_model_dir=best_model_dir,
            tokenizer=self.tokenizer
        )
        
        # 配置训练参数
        training_args = GRPOConfig(
            # 基本训练参数
            output_dir=self.config.get('output.output_dir'),                                    # 输出目录
            per_device_train_batch_size=self.config.get('training.per_device_train_batch_size',2),  # 每设备训练批次大小
            per_device_eval_batch_size=self.config.get('training.per_device_eval_batch_size', 4),   # 每设备验证批次大小
            gradient_accumulation_steps=self.config.get('training.gradient_accumulation_steps'),    # 梯度累积步数
            learning_rate=self.config.get('training.learning_rate'),                               # 学习率
            num_train_epochs=self.config.get('training.num_train_epochs'),                         # 训练轮数
            logging_steps=self.config.get('output.logging_steps'),                                 # 日志记录步数
            
            # GRPO特有参数
            num_generations=self.config.get('grpo.num_generations', 8),    # 每个prompt生成的回复数量
            temperature=self.config.get('grpo.temperature', 0.8),          # 生成温度（控制随机性）
            top_p=self.config.get('grpo.top_p', 0.9),                     # 核采样参数
            epsilon=self.config.get('grpo.epsilon', 0.2),                  # PPO裁剪参数
            beta=self.config.get('grpo.beta', 0.0),                       # KL散度惩罚系数
            
            # 修复生成长度限制
            max_completion_length=self.config.get('data.max_completion_length', 512),  # 最大生成长度
            
            # Checkpoint策略
            save_strategy="steps",                                          # 按步数保存
            save_steps=self.config.get('output.save_steps', 500),         # 保存间隔
            save_total_limit=self.config.get('output.save_total_limit', 3), # 最大保存数量
            
            # 评估策略 
            eval_strategy="steps",                                          # 按步数评估
            eval_steps=self.config.get('output.eval_steps', 100),         # 评估间隔
            
            # 不使用传统的最佳模型加载（我们自己管理）
            load_best_model_at_end=False,                                  # 训练结束时不加载最佳模型
            metric_for_best_model=None,                                    # 不指定最佳模型指标
            
            # 其他参数
            warmup_steps=self.config.get('training.warmup_steps', 50),     # 预热步数
            max_grad_norm=self.config.get('training.max_grad_norm', 1.0),  # 梯度裁剪
            dataloader_num_workers=self.config.get('training.dataloader_num_workers', 0),  # 数据加载线程数
            bf16=True,                                                      # 使用bfloat16精度
            remove_unused_columns=False,                                   # 保留所有数据列
            deepspeed=self.deepspeed_config,                               # DeepSpeed配置
            report_to=["tensorboard"],                                     # 使用TensorBoard监控
            logging_dir=os.path.join(self.config.get('output.output_dir'), "logs"),  # 日志目录
        )
        
        # 创建GRPO训练器
        self.trainer = GRPOTrainer(
            model=self.model,                    # 策略模型
            args=training_args,                  # 训练参数
            train_dataset=self.dataset,          # 训练数据集
            eval_dataset=self.eval_dataset,      # 验证数据集
            processing_class=self.tokenizer,     # 文本处理器（分词器）
            reward_funcs=[self.reward_function], # 奖励函数列表
            callbacks=[early_stopping_callback] # 回调函数列表
        )
        
        self.logger.info("GRPO训练器设置完成")
    
    def train(self, resume_from_checkpoint=None):
        """
        执行GRPO训练
        
        参数：
            resume_from_checkpoint: 从指定checkpoint恢复训练
        
        作用：
        1. 按顺序初始化所有组件
        2. 执行完整的训练流程
        3. 保存最终模型
        4. 处理训练过程中的异常
        
        训练流程：
        1. 记录训练开始
        2. 初始化各个组件
        3. 执行GRPO训练
        4. 保存模型和分词器
        5. 记录训练结果
        """
        try:
            # 记录训练开始
            self.logger.log_training_start(self.config)
            
            # 按顺序初始化所有组件
            self.setup_tokenizer()      # 1. 设置分词器
            self.setup_model()          # 2. 加载模型
            self.setup_dataset()        # 3. 准备数据
            self.setup_reward_function() # 4. 创建奖励函数
            self.setup_trainer()        # 5. 配置训练器
            
            # 开始训练
            self.logger.info("开始GRPO训练...")
            self.trainer.train(resume_from_checkpoint=resume_from_checkpoint)
            
            # 保存最终模型
            self.logger.info("保存模型...")
            self.trainer.save_model()  # 保存模型权重
            self.tokenizer.save_pretrained(self.config.get('output.output_dir'))  # 保存分词器
            
            # 记录训练成功
            self.logger.log_training_end(success=True)
            
        except Exception as e:
            # 处理训练失败
            self.logger.error(f"训练失败: {str(e)}")
            self.logger.log_training_end(success=False)
            raise  # 重新抛出异常


def main():
    """
    主函数：程序入口点
    
    作用：
    1. 解析命令行参数
    2. 加载配置
    3. 创建训练器
    4. 启动训练
    
    支持的命令行参数：
    - --deepspeed: DeepSpeed配置文件路径
    - --resume_from_checkpoint: 从checkpoint恢复训练
    """
    # 解析命令行参数
    parser = argparse.ArgumentParser()
    parser.add_argument('--deepspeed', type=str, help='DeepSpeed配置文件路径')
    parser.add_argument('--resume_from_checkpoint', type=str, default=None, 
                       help='从checkpoint恢复训练')
    args = parser.parse_known_args()[0]  # 只取已知参数
    
    # 加载配置和创建训练器
    config = ProjectConfig()  # 加载项目配置
    trainer = GRPOTraining(config, args.deepspeed)  # 创建训练器
    
    # 启动训练
    trainer.train(resume_from_checkpoint=args.resume_from_checkpoint)


# 程序入口点
if __name__ == "__main__":
    main()
```

config.json    主配置文件


```json
{
  "model": {
    "model_name": "/shared/DeepSeek-R1-Distill-Qwen-7B_028/best_complete_model_05261653_028",
    "reward_model_path": "/shared/Skywork-Reward_checkpoint-1000/checkpoint-1000",
    "lora_model_path": null
  },
  "data": {
    "train_data_path": "/shared/grpo_financial_tuning/data/grpo_prompts_dataset_5k.jsonl",
    "eval_data_path": "/shared/grpo_financial_tuning/data/eval_prompts_dataset.jsonl",
    "max_prompt_length": 512,
    "max_completion_length": 1024
  },
  "training": {
    "learning_rate": 1e-6,
    "num_train_epochs": 3,
    "per_device_train_batch_size": 4,
    "per_device_eval_batch_size": 4,
    "gradient_accumulation_steps": 4,
    "warmup_steps": 10,
    "bf16": true,
    "gradient_checkpointing": false,
    "max_grad_norm": 1.0
  },
  "grpo": {
    "num_generations": 4,
    "temperature": 0.7,
    "top_p": 0.9,
    "epsilon": 0.2,
    "beta": 0.0,
    "use_liger_loss": true
  },
  "output": {
    "output_dir": "./output/",
    "logging_steps": 1,
    "save_steps": 10,
    "eval_steps": 10,
    "load_best_model_at_end": true,
    "save_total_limit": 1
  },
  "logging": {
    "log_level": "INFO",
    "use_wandb": false,
    "wandb_project": "grpo-financial-tuning"
  }
}
```

```json
{
  "model": {
    // 基座模型路径 - 这是已经过SFT训练的DeepSeek-R1模型
    // 作用：作为GRPO训练的起始点，提供基础的对话能力
    "model_name": "/shared/DeepSeek-R1-Distill-Qwen-7B_028/best_complete_model_05261653_028",
    
    // 奖励模型路径 - 用于评估生成文本质量的专门模型
    // 作用：为GRPO训练提供奖励信号，指导模型优化方向
    // 注意：必须与策略模型使用相同的文本格式（LLaMA-3格式）
    "reward_model_path": "/shared/Skywork-Reward_checkpoint-1000/checkpoint-1000",
    
    // LoRA适配器路径 - 参数高效微调适配器
    // null表示不使用LoRA，进行全参数微调
    // 如果启用，可以大幅减少可训练参数数量（通常<1%）
    "lora_model_path": null
  },
  
  "data": {
    // 训练数据路径 - GRPO训练的prompt数据集
    // 格式：每行一个JSON，包含{"prompt": "问题内容"}
    "train_data_path": "/shared/grpo_financial_tuning/data/grpo_prompts_dataset_5k.jsonl",
    
    // 验证数据路径 - 用于训练过程中的模型评估
    // 作用：监控训练进度，触发最佳模型保存
    "eval_data_path": "/shared/grpo_financial_tuning/data/eval_prompts_dataset.jsonl",
    
    // 最大输入长度 - prompt的最大token数
    // 作用：控制输入序列长度，影响显存占用和训练速度
    // 512对应约400-500个中文字符
    "max_prompt_length": 512,
    
    // 最大生成长度 - 模型单次生成的最大token数
    // 作用：控制输出序列长度，直接影响显存占用
    // 1024对应约800-1000个中文字符
    "max_completion_length": 1024
  },
  
  "training": {
    // 学习率 - 控制参数更新的步长
    // 1e-6 = 0.000001，这是一个很小的学习率
    // 原因：GRPO训练需要谨慎的参数更新，避免策略崩溃
    "learning_rate": 1e-6,
    
    // 训练轮数 - 完整遍历训练数据的次数
    // 3轮对于强化学习训练通常足够，过多可能导致过拟合
    "num_train_epochs": 3,
    
    // 每GPU训练批次大小 - 每个GPU同时处理的样本数
    // 4是在显存限制下的平衡选择（7B模型+A800-80GB）
    "per_device_train_batch_size": 4,
    
    // 每GPU验证批次大小 - 验证时每个GPU处理的样本数
    // 通常与训练批次大小相同或略大
    "per_device_eval_batch_size": 4,
    
    // 梯度累积步数 - 多少步后进行一次参数更新
    // 实际批次大小 = per_device_batch_size × gradient_accumulation_steps × GPU数量
    // 这里：4 × 4 × 2 = 32的有效批次大小
    "gradient_accumulation_steps": 4,
    
    // 预热步数 - 学习率从0逐渐增加到目标值的步数
    // 10步预热有助于训练初期的稳定性
    "warmup_steps": 10,
    
    // 混合精度训练 - 使用bfloat16减少显存占用
    // true启用，可节省约50%显存，对7B模型几乎无精度损失
    "bf16": true,
    
    // 梯度检查点 - 用时间换空间的内存优化技术
    // false表示不启用，因为当前配置显存充足
    "gradient_checkpointing": false,
    
    // 梯度裁剪阈值 - 防止梯度爆炸
    // 1.0是常用值，当梯度范数超过此值时进行缩放
    "max_grad_norm": 1.0
  },
  
  "grpo": {
    // 【核心参数】每个prompt生成的回复数量
    // 4表示为每个问题生成4个不同的回答
    // 作用：提供组内比较的样本，是GRPO算法的核心
    // 影响：直接影响显存占用（4倍）和训练效果
    "num_generations": 4,
    
    // 生成温度 - 控制文本生成的随机性
    // 0.7是平衡值：0=确定性，1=高随机性
    // 作用：确保4个生成样本有足够的多样性进行比较
    "temperature": 0.7,
    
    // 核采样参数 - 控制候选token的范围
    // 0.9表示从累积概率90%的token中采样
    // 作用：与temperature配合控制生成质量和多样性
    "top_p": 0.9,
    
    // PPO裁剪参数 - 控制策略更新的幅度
    // 0.2是标准值，防止策略更新过大导致训练不稳定
    "epsilon": 0.2,
    
    // KL散度惩罚系数 - 控制新策略与原策略的差异
    // 0.0表示不使用KL惩罚，GRPO通常不需要此项
    "beta": 0.0,
    
    // Liger优化 - 使用优化的损失函数实现
    // true启用，可以提升训练速度和内存效率
    "use_liger_loss": true
  },
  
  "output": {
    // 输出目录 - 模型、日志、检查点的保存位置
    "output_dir": "./output/",
    
    // 日志记录频率 - 每多少步记录一次训练指标
    // 1表示每步都记录，便于密切监控GRPO训练过程
    "logging_steps": 1,
    
    // 模型保存频率 - 每多少步保存一次检查点
    // 10步保存一次，平衡存储空间和恢复粒度
    "save_steps": 10,
    
    // 验证频率 - 每多少步进行一次模型验证
    // 10步验证一次，及时发现最佳模型
    "eval_steps": 10,
    
    // 训练结束时是否加载最佳模型
    // true表示自动加载验证效果最好的模型
    "load_best_model_at_end": true,
    
    // 最大保存检查点数量 - 控制磁盘空间占用
    // 1表示只保留最新的检查点，节省存储空间
    "save_total_limit": 1
  },
  
  "logging": {
    // 日志级别 - 控制日志输出的详细程度
    // INFO级别输出关键信息，不包含调试细节
    "log_level": "INFO",
    
    // 是否使用Weights & Biases进行实验跟踪
    // false表示不使用，避免外部依赖
    "use_wandb": false,
    
    // W&B项目名称 - 如果启用wandb时使用
    "wandb_project": "grpo-financial-tuning"
  }
}
```

deepspeed_zero2.json  DeepSpeed分布式配置

```json
{
    "zero_optimization": {
        "stage": 2,
        "offload_optimizer": {
            "device": "cpu",
            "pin_memory": true
        },
        "allgather_partitions": true,
        "allgather_bucket_size": 2e8,
        "overlap_comm": true,
        "reduce_scatter": true,
        "reduce_bucket_size": 2e8,
        "contiguous_gradients": true
    },
    "gradient_accumulation_steps": "auto",
    "gradient_clipping": "auto",
    "steps_per_print": 10,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "wall_clock_breakdown": false,
    "bf16": {
        "enabled": true
    },
    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto"  
        }
    },
    "scheduler": {
        "type": "WarmupLR",
        "params": {
            "warmup_min_lr": "auto",
            "warmup_max_lr": "auto",
            "warmup_num_steps": "auto"
        }
    }
} 
```

```json
{
    "zero_optimization": {
        // ZeRO优化阶段 - 控制内存优化的激进程度
        // Stage 2：分片优化器状态 + 梯度，保留完整模型参数
        // 选择原因：平衡内存节省和通信开销，适合多机训练
        "stage": 2,
        
        "offload_optimizer": {
            // 优化器状态卸载到CPU内存
            // 作用：释放GPU显存，支持更大模型或批次
            // 代价：增加CPU-GPU数据传输开销
            "device": "cpu",
            
            // 使用固定内存 - 避免CPU内存分配延迟
            // true可以提升CPU-GPU传输效率
            "pin_memory": true
        },
        
        // 启用allgather分区 - 优化参数聚合过程
        // true可以减少通信次数，提升训练效率
        "allgather_partitions": true,
        
        // allgather操作的缓冲区大小 - 2e8 = 200MB
        // 作用：控制通信的批次大小，影响内存和带宽利用率
        // 较大值减少通信次数但增加内存占用
        "allgather_bucket_size": 2e8,
        
        // 重叠通信和计算 - 异步执行提升效率
        // true表示在计算的同时进行参数通信
        "overlap_comm": true,
        
        // 启用reduce_scatter优化 - 梯度聚合优化
        // true可以提升多GPU间梯度同步效率
        "reduce_scatter": true,
        
        // reduce_scatter的缓冲区大小 - 2e8 = 200MB
        // 与allgather_bucket_size配合，平衡内存和通信
        "reduce_bucket_size": 2e8,
        
        // 连续梯度存储 - 优化内存布局
        // true可以提升梯度通信效率，减少内存碎片
        "contiguous_gradients": true
    },
    
    // 梯度累积步数 - 由训练配置自动设置
    // "auto"表示从config.json的gradient_accumulation_steps读取
    "gradient_accumulation_steps": "auto",
    
    // 梯度裁剪 - 由训练配置自动设置
    // "auto"表示从config.json的max_grad_norm读取
    "gradient_clipping": "auto",
    
    // 打印训练状态的频率 - 每10步打印一次
    // 用于监控训练进度和性能指标
    "steps_per_print": 10,
    
    // 全局批次大小 - 自动计算
    // 实际值 = per_device_batch_size × gradient_accumulation_steps × world_size
    // 这里：4 × 4 × 2 = 32
    "train_batch_size": "auto",
    
    // 每GPU微批次大小 - 自动从配置读取
    // 对应config.json中的per_device_train_batch_size
    "train_micro_batch_size_per_gpu": "auto",
    
    // 是否启用详细的时间分析 - 性能调试工具
    // false表示不启用，避免额外开销
    "wall_clock_breakdown": false,
    
    "bf16": {
        // 启用bfloat16混合精度训练
        // 与config.json中的bf16参数对应
        // 作用：减少显存占用，提升训练速度
        "enabled": true
    },
    
    "optimizer": {
        // 优化器类型 - AdamW是当前LLM训练的标准选择
        // 相比Adam增加了权重衰减的正则化
        "type": "AdamW",
        
        "params": {
            // 学习率 - 从config.json自动读取
            // 对应training.learning_rate = 1e-6
            "lr": "auto",
            
            // Adam的momentum参数 - 自动使用默认值[0.9, 0.999]
            // 控制梯度的指数移动平均
            "betas": "auto",
            
            // 数值稳定性参数 - 自动使用默认值1e-8
            // 防止除零错误
            "eps": "auto",
            
            // 权重衰减系数 - 自动使用默认值0.01
            // L2正则化，防止过拟合
            "weight_decay": "auto"  
        }
    },
    
    "scheduler": {
        // 学习率调度器类型 - 预热后线性衰减
        // WarmupLR先增加后保持学习率
        "type": "WarmupLR",
        
        "params": {
            // 预热最小学习率 - 通常为0
            "warmup_min_lr": "auto",
            
            // 预热最大学习率 - 等于目标学习率1e-6
            "warmup_max_lr": "auto",
            
            // 预热步数 - 从config.json读取warmup_steps = 10
            "warmup_num_steps": "auto"
        }
    }
}
```

run_training.sh   训练启动脚本

```bash
#!/bin/bash

# ==================== GRPO训练脚本 ====================
# 作用：支持nohup后台运行和多机多卡训练的启动脚本
# 功能：环境配置、进程管理、分布式训练、错误处理

# 设置严格模式：任何命令失败都会导致脚本退出
set -e

# ==================== 多机多卡配置 ====================
# 定义集群节点配置：IP地址到GPU数量的映射
declare -A NODES=(
    ["10.60.68.220"]=1    # 主节点：拥有2张GPU卡
    ["10.60.98.173"]=1    # 从节点：拥有1张GPU卡
)
MASTER_ADDR="10.60.68.220"  # 主节点IP地址

# ==================== 脚本参数解析 ====================
# 初始化命令行参数变量
USE_NOHUP=false        # 是否使用nohup后台运行
STOP_TRAINING=false    # 是否停止训练
CHECKPOINT_PATH=""     # checkpoint恢复路径
USE_MULTINODE=false    # 是否使用多机训练

# 解析命令行参数
while [[ $# -gt 0 ]]; do
    case $1 in
        --nohup) USE_NOHUP=true; shift ;;           # 启用后台运行
        --stop) STOP_TRAINING=true; shift ;;        # 停止训练
        --resume) CHECKPOINT_PATH="$2"; shift 2 ;;  # 从checkpoint恢复
        --multinode) USE_MULTINODE=true; shift ;;   # 启用多机训练
        *) echo "用法: $0 [--nohup] [--stop] [--resume <checkpoint_path>] [--multinode]"; exit 1 ;;
    esac
done

# ==================== 工具函数定义 ====================

create_hostfile() {
    """
    创建DeepSpeed hostfile
    
    作用：
    1. 生成DeepSpeed分布式训练需要的主机文件
    2. 指定每个节点的GPU数量
    3. 支持多机多卡训练配置
    
    hostfile格式：
    IP地址 slots=GPU数量
    """
    echo "创建DeepSpeed hostfile..."
    > ./hostfile  # 清空hostfile
    
    # 遍历所有节点配置
    for node_ip in "${!NODES[@]}"; do
        gpu_count=${NODES[$node_ip]}
        echo "$node_ip slots=$gpu_count" >> ./hostfile
    done
    
    echo "Hostfile创建完成:"
    cat ./hostfile  # 显示hostfile内容
}

stop_all_training() {
    """
    停止所有节点的训练进程
    
    作用：
    1. 在所有节点上查找并终止训练进程
    2. 支持单机和多机环境
    3. 确保进程完全清理
    
    进程识别：
    - 通过进程命令行参数识别训练进程
    - 使用pkill命令批量终止进程
    """
    echo "停止所有节点的训练进程..."
    
    for node_ip in "${!NODES[@]}"; do
        if [[ "$node_ip" == "$MASTER_ADDR" ]]; then
            # 在主节点上直接执行
            pkill -f "deepspeed.*train.py" 2>/dev/null || true
        else
            # 通过SSH在从节点上执行
            ssh ubuntu@$node_ip "pkill -f 'deepspeed.*train.py' 2>/dev/null || true" &
        fi
    done
    
    wait    # 等待所有后台任务完成
    sleep 3 # 等待进程完全退出
    echo "所有节点训练进程清理完成"
}

# ==================== 停止训练功能 ====================
if [ "$STOP_TRAINING" = true ]; then
    if [ "$USE_MULTINODE" = true ]; then
        # 多机环境：停止所有节点
        stop_all_training
    else
        # 单机环境：只停止本地进程
        echo "正在停止GRPO训练进程..."
        TRAIN_PIDS=$(ps aux | grep "deepspeed.*train.py" | grep -v grep | awk '{print $2}')
        
        if [ -z "$TRAIN_PIDS" ]; then
            echo "未找到运行中的训练进程"
        else
            echo "停止进程: $TRAIN_PIDS"
            for pid in $TRAIN_PIDS; do
                kill -TERM $pid  # 发送终止信号
            done
            echo "训练进程已停止"
        fi
    fi
    exit 0
fi

# ==================== 环境准备 ====================

# 切换到脚本所在目录
cd "$(dirname "$0")"
    
# 创建日志目录（带时间戳）
LOG_DIR="./logs/$(date +%Y%m%d_%H%M%S)"
mkdir -p "$LOG_DIR"

# ==================== 环境变量设置 ====================
# GPU和CUDA配置
export CUDA_VISIBLE_DEVICES=0,1        # 指定可见的GPU设备
export NCCL_DEBUG=WARN                 # NCCL调试级别
export NCCL_IB_DISABLE=1               # 禁用InfiniBand
export NCCL_IGNORE_DISABLED_P2P=1      # 忽略P2P通信问题

# Python环境配置
export PYTHONPATH=$PYTHONPATH:$(pwd)   # 添加当前目录到Python路径
export PDSH_RCMD_TYPE=ssh              # 设置远程命令执行方式

# ==================== NCCL优化配置 ====================
# NCCL（NVIDIA Collective Communication Library）是GPU间通信库
# 这些配置用于解决多机多卡训练中的通信问题
export NCCL_BUFFSIZE=524288             # 强制使用524288字节缓冲区
export NCCL_MAX_NCHANNELS=1             # 限制最大通道数量
export NCCL_MIN_NCHANNELS=1             # 限制最小通道数量
export NCCL_PROTO=Simple                # 使用简单通信协议
export NCCL_ALGO=Ring                   # 使用Ring通信算法
export NCCL_TREE_THRESHOLD=0            # 禁用树形通信
export NCCL_NET_GDR_LEVEL=0             # 禁用GPU Direct RDMA
export NCCL_SOCKET_IFNAME=eth0          # 强制使用eth0网卡

# 创建输出目录
mkdir -p "./output/"

# ==================== 训练命令构建 ====================

build_train_command() {
    """
    构建训练命令
    
    参数：无（使用全局变量）
    
    返回：完整的训练命令字符串
    
    作用：
    1. 根据运行模式构建不同的命令
    2. 支持单机和多机训练
    3. 处理checkpoint恢复
    4. 动态配置参数
    """
    local cmd
    local current_dir=$(pwd)
    
    if [ "$USE_MULTINODE" = true ]; then
        # 多机训练命令
        cmd="cd $current_dir && deepspeed --hostfile=hostfile $current_dir/train.py --deepspeed $current_dir/configs/deepspeed_zero2.json"
    else
        # 单机训练命令
        cmd="deepspeed --num_gpus=2 train.py --deepspeed configs/deepspeed_zero2.json"
    fi
    
    # 添加checkpoint恢复参数
    if [ -n "$CHECKPOINT_PATH" ]; then
        cmd="$cmd --resume_from_checkpoint $CHECKPOINT_PATH"
        echo "从checkpoint恢复训练: $CHECKPOINT_PATH" >&2
    fi
    
    echo "$cmd"
}

# ==================== 训练执行函数 ====================

run_training() {
    """
    执行训练
    
    作用：
    1. 激活Python环境
    2. 准备多机环境（如需要）
    3. 执行训练命令
    4. 处理训练结果
    
    返回：0表示成功，1表示失败
    """
    echo "开始GRPO训练 - $(date)"

    # 激活conda环境
    source ~/miniconda3/etc/profile.d/conda.sh
    conda activate reward3 
    
    # 多机模式需要先准备环境
    if [ "$USE_MULTINODE" = true ]; then
        stop_all_training  # 清理旧进程
        create_hostfile    # 创建hostfile
    fi
    
    # 构建并执行训练命令
    local train_cmd=$(build_train_command)
    if eval "$train_cmd"; then
        echo "训练成功完成 - $(date)"
        return 0
    else
        echo "训练失败 - $(date)"
        return 1
    fi
}

# ==================== 主执行逻辑 ====================

if [ "$USE_NOHUP" = true ]; then
    # ==================== 后台运行模式 ====================
    echo "后台运行训练，日志: $LOG_DIR/training.log"
    echo "查看进度: tail -f $LOG_DIR/training.log"
    echo "停止训练: pkill -f 'deepspeed.*train.py'"
    
    # 多机模式需要先准备环境
    if [ "$USE_MULTINODE" = true ]; then
        stop_all_training
        create_hostfile
    fi
    
    # 构建完整的后台命令
    train_cmd=$(build_train_command)
    nohup bash -c "source ~/miniconda3/etc/profile.d/conda.sh && conda activate reward3 && $train_cmd" > "$LOG_DIR/training.log" 2>&1 &
    
    # 记录进程ID
    TRAIN_PID=$!
    echo "训练进程已启动 (PID: $TRAIN_PID)"
    echo $TRAIN_PID > "$LOG_DIR/train.pid"
else
    # ==================== 前台运行模式 ====================
    # 重定向输出到日志文件同时显示在控制台
    exec > >(tee -a "$LOG_DIR/training.log")
    exec 2>&1
    run_training
fi
```

执行训练命令

```bash
# 后端执行多机多卡训练
./run_training.sh --nohup --multinode
# 从checkpoint节点继续训练
./run_training.sh --nohup --multinode --resume ./output/checkpoint-40
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250709143027262.png" width=100%></div>

**GRPO工作原理**：
1. 给模型一个金融问题
2. 模型生成4个不同的回答
3. 奖励模型给这4个回答打分
4. 选择高分回答，惩罚低分回答
5. 调整模型参数，让它更倾向于生成高分回答

### 参数波动的实际影响

#### 1. **Reward波动：18.80→20.89→16.69→19.34**

**具体影响**：
- **20.89时**：模型生成的金融回答质量很高，专业性强，逻辑清晰
- **16.69时**：模型生成的回答质量下降，可能出现：
  - 回答不够专业
  - 逻辑混乱
  - 偏离金融主题
- **19.34时**：质量回升，模型重新找到了生成好回答的方向

**为什么会波动**？
- 这是强化学习的正常现象
- 模型在"探索"（尝试新的回答方式）和"利用"（使用已知的好方法）之间平衡
- 就像学生做题，有时会尝试新方法导致暂时成绩下降

#### 2. **Reward_std(Reward标准差)波动：2.08→3.21→2.76**

**具体影响**：
- **2.08时**：4个回答的质量很接近，都比较稳定
  - 回答1：19分，回答2：18分，回答3：19分，回答4：18分
- **3.21时**：4个回答质量差异很大，不稳定
  - 回答1：22分，回答2：15分，回答3：20分，回答4：16分
- **2.76时**：质量差异缩小，趋于稳定

**实际意义**：
- 低std = 模型回答质量稳定，用户体验一致
- 高std = 模型回答质量不稳定，有时好有时差

#### 3. **Loss(损失函数)波动：-0.0231→0.0336→-0.0255**

GRPO的策略损失，负值表示策略改进方向

**具体影响**：
- **负值（-0.0231）**：模型正在朝着正确方向学习
  - 就像学生答题正确率在提高
- **正值（0.0336）**：模型暂时"迷失"了方向
  - 就像学生暂时做错了题
- **负值（-0.0255）**：模型重新找到正确方向

#### 4. **Grad_norm(梯度范数)波动：0.76→0.84→0.72**

**具体影响**：
- **0.76**：模型参数调整幅度适中，学习稳定
- **0.84**：参数调整幅度稍大，学习激进
- **0.72**：参数调整幅度较小，学习保守

**类比理解**：
- 就像调音师调钢琴，数值越大调整幅度越大
- 太大 = 可能调过头
- 太小 = 调整太慢
- 0.7-0.8 = 调整幅度刚好



### 实际用户体验对比

#### **Step 3（表现最好）：reward=20.89, reward_std=2.62**
用户问："如何分析一只股票的投资价值？"

模型回答质量：
- 回答专业性强
- 逻辑清晰
- 4个回答都比较好，质量接近

#### **Step 8（表现最差）：reward=16.69, reward_std=3.02**
同样的问题，模型回答质量：
- 回答可能偏离主题
- 专业术语使用不当
- 4个回答质量差异很大，有好有坏

#### **Step 11（回升）：reward=19.34, reward_std=2.76**
模型回答质量：
- 专业性恢复
- 逻辑性改善
- 质量趋于稳定



<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250706210948443.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250706185011783.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250709161020433.png" width=100%></div>

tensorboard 启动命令

tensorboard --logdir=./output/logs/ --port=6006 --bind_all

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250709144323080.png" width=100%></div>

访问对应服务器外部访问链接地址 http://服务器外部IP:6006/#timeseries

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250709144507491.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250709144521529.png" width=100%></div>

# 6. 模型测试

test.py   模型测试脚本单节点最简单测试

```python
# ==================== 模型测试脚本 ====================
# 作用：加载训练好的GRPO模型并进行推理测试
# 用途：验证模型是否正常工作，测试模型的推理能力

import torch                                    # PyTorch深度学习框架
from transformers import AutoTokenizer, AutoModelForCausalLM  # Hugging Face模型和分词器

def test_model():
    """
    测试已训练的GRPO模型
    
    功能：
    1. 加载保存的模型和分词器
    2. 使用金融领域的复杂问题进行推理测试
    3. 验证模型的文本生成能力
    """
    # 设置模型路径 - 可以是本地路径或远程模型路径
    model_path = "./output/best_model"  # 默认使用本地最佳模型
    # model_path = "/shared/DeepSeek-R1-Distill-Qwen-7B_028/best_complete_model_05261653_028"  # 备用路径
    
    print("加载分词器...")
    # 加载分词器 - 负责将文本转换为模型可理解的数字序列
    tokenizer = AutoTokenizer.from_pretrained(
        model_path,                    # 模型路径
        trust_remote_code=True         # 允许执行远程代码（某些模型需要）
    )
    print("分词器加载成功")
    
    print("加载模型...")
    # 加载语言模型 - 用于文本生成
    model = AutoModelForCausalLM.from_pretrained(
        model_path,                    # 模型路径
        torch_dtype=torch.bfloat16,    # 使用bfloat16精度节省显存
        device_map="auto",             # 自动分配设备（GPU/CPU）
        trust_remote_code=True         # 允许执行远程代码
    )
    print("模型加载成功")
    
    print("测试推理...")
    # 构造测试用的金融问题 - 这是一个复杂的金融分析问题
    prompt = "你是一个专业的金融领域分析师请对以下问题进行详细解答：在周二的交易中，First Trust纳斯达克网络安全ETF(CIBR)表现优于其他ETF，当日上涨约1.3%。该ETF中表现尤为强劲的成分股包括Sentinelone(上涨约5.3%)和Okta(上涨约3.8%)。与此同时，Invesco太阳能ETF(TAN)表现逊于其他ETF，周二下午交易时段下跌约3.7%。该ETF中表现最弱的成分股包括Maxeon Solar Technologies(下跌约7.8%)和Solaredge Technologies(下跌约6.7%)。基于这些市场表现数据，请分析：1) 网络安全和太阳能行业ETF表现差异可能反映出的宏观经济或行业特定因素；2) 成分股价格变动与ETF整体表现之间的传导机制；3) 这种行业间表现差异可能对投资者资产配置策略产生的影响；4) 如何利用ETF成分股的价格离散度来构建潜在的投资组合策略。请详细说明你的分析框架和逻辑推理过程。"
    
    # 将文本转换为模型输入格式
    inputs = tokenizer.encode(
        prompt,                        # 输入文本
        return_tensors="pt"           # 返回PyTorch张量格式
    ).to(model.device)                # 移动到模型所在设备
    
    # 使用模型生成回复
    with torch.no_grad():             # 禁用梯度计算，节省内存
        outputs = model.generate(
            inputs,                    # 输入序列
            max_new_tokens=2048,       # 最大生成长度
            temperature=0.7,           # 控制生成的随机性（0-1，越高越随机）
            do_sample=True,            # 启用采样生成
            pad_token_id=tokenizer.eos_token_id  # 设置填充token
        )
    
    # 将生成的数字序列转换回文本
    result = tokenizer.decode(
        outputs[0],                   # 取第一个生成结果
        skip_special_tokens=True      # 跳过特殊标记
    )
    
    print("推理成功")
    print("结果:", result)
    print("测试完成，模型正常工作")

# 程序入口点
if __name__ == "__main__":
    test_model()  # 执行模型测试
```

执行命令 python test.py

GRPO微调后模型

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250704000439137.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250704000503767.png" width=100%></div>

GRPO微调前

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250704000631336.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250704000656072.png" width=100%></div>

multigpu_evaluator.py   多GPU评估工具

```python
#!/usr/bin/env python3
"""
多机多卡奖励模型评分工具 - 使用DeepSpeed
这个工具用于在多个GPU上同时评测GRPO训练后的模型质量
主要功能：给定一些问题，让模型生成回答，然后用奖励模型给回答打分
"""
import torch
import json
import deepspeed
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import argparse
import os

class PromptDataset(Dataset):
    """
    Prompt数据集类
    作用：将输入的问题列表转换为PyTorch可以处理的数据集格式
    每个问题会复制num_generations次，因为每个问题要生成多个回答
    """
    def __init__(self, prompts: list, num_generations: int = 4):
        """
        初始化数据集
        prompts: 问题列表，例如["如何分析股票？", "什么是基金？"]
        num_generations: 每个问题生成几个回答，默认4个
        """
        self.data = []
        # 将每个问题复制num_generations次
        # 例如：1个问题 × 4次生成 = 4个数据项
        for prompt in prompts:
            for _ in range(num_generations):
                self.data.append(prompt)
    
    def __len__(self):
        """返回数据集总大小"""
        return len(self.data)
    
    def __getitem__(self, idx):
        """根据索引返回对应的问题"""
        return self.data[idx]

class MultiGPURewardEvaluator:
    """
    多GPU奖励模型评分器
    作用：在多个GPU上同时运行模型评测，提高评测效率
    包含两个模型：主模型（生成回答）和奖励模型（给回答打分）
    """
    def __init__(self, model_path: str, reward_model_path: str):
        """
        初始化多GPU评分器
        model_path: 主模型路径（GRPO训练后的模型）
        reward_model_path: 奖励模型路径（用于给回答打分）
        """
        self.model_path = model_path
        self.reward_model_path = reward_model_path
        
        # 初始化DeepSpeed分布式环境
        # DeepSpeed是微软开发的深度学习优化库，支持多GPU训练和推理
        deepspeed.init_distributed()
        
        # 获取当前进程的GPU编号和总GPU数量
        # LOCAL_RANK: 当前进程在本机上的GPU编号（0, 1, 2...）
        # WORLD_SIZE: 总的GPU数量
        self.local_rank = int(os.environ.get('LOCAL_RANK', 0))
        self.world_size = int(os.environ.get('WORLD_SIZE', 1))
        
        # 设置当前进程使用的GPU设备
        torch.cuda.set_device(self.local_rank)
        self.device = torch.device(f'cuda:{self.local_rank}')
        
        # 加载和配置模型
        self.setup_models()
    
    def setup_models(self):
        """
        设置和加载模型
        作用：加载主模型和奖励模型，并使用DeepSpeed进行优化
        """
        # 只在主进程（GPU 0）上打印信息，避免重复输出
        if self.local_rank == 0:
            print(f"加载模型: {self.model_path}")
        
        # 加载主模型的分词器
        # 分词器负责将文本转换为模型可以理解的数字序列
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path, trust_remote_code=True)
        # 如果分词器没有padding token，使用结束token作为padding
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        # 加载主模型（用于生成回答）
        # torch_dtype=torch.bfloat16: 使用半精度浮点数，节省显存
        model = AutoModelForCausalLM.from_pretrained(
            self.model_path, torch_dtype=torch.bfloat16, trust_remote_code=True
        )
        
        # 使用DeepSpeed初始化主模型
        # DeepSpeed会自动处理模型的分布式部署和内存优化
        self.model, _, _, _ = deepspeed.initialize(
            model=model,
            config={
                "train_batch_size": 2,                    # 训练批次大小
                "train_micro_batch_size_per_gpu": 1,      # 每个GPU的微批次大小
                "gradient_accumulation_steps": 1,          # 梯度累积步数
                "bf16": {"enabled": True},                 # 启用bfloat16精度
                "zero_optimization": {"stage": 0}          # ZeRO优化级别0（不分割参数）
            }
        )
        
        # 加载奖励模型 - 参考dataset.py的实现方式
        if self.local_rank == 0:
            print(f"加载奖励模型: {self.reward_model_path}")
        
        # 加载奖励模型的分词器
        self.reward_tokenizer = AutoTokenizer.from_pretrained(self.reward_model_path, trust_remote_code=True)
        if self.reward_tokenizer.pad_token is None:
            self.reward_tokenizer.pad_token = self.reward_tokenizer.eos_token
        
        # 加载奖励模型（用于给回答打分）
        # num_labels=1: 输出一个分数值
        # use_cache=False: 不使用缓存，节省内存
        reward_model = AutoModelForSequenceClassification.from_pretrained(
            self.reward_model_path, num_labels=1, torch_dtype=torch.bfloat16, 
            trust_remote_code=True, use_cache=False
        )
        
        # 使用DeepSpeed初始化奖励模型
        self.reward_model, _, _, _ = deepspeed.initialize(
            model=reward_model,
            config={
                "train_batch_size": 2,
                "train_micro_batch_size_per_gpu": 1,
                "gradient_accumulation_steps": 1,
                "bf16": {"enabled": True},
                "zero_optimization": {"stage": 0}
            }
        )
    
    def generate_response(self, prompt: str) -> str:
        """
        生成回复
        作用：给定一个问题，使用主模型生成一个回答
        prompt: 输入的问题文本
        返回：模型生成的回答文本
        """
        # 将问题格式化为对话格式
        # 这里使用系统提示词让模型扮演金融分析师角色
        messages = [
            {"role": "system", "content": "你是一个专业的金融分析师，擅长投资分析和风险评估。请基于提供的信息进行详细的分析和推理。"},
            {"role": "user", "content": prompt}
        ]
        
        # 使用分词器的聊天模板功能格式化对话
        # add_generation_prompt=True: 添加生成提示符
        formatted_prompt = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
        # 将文本转换为模型输入格式
        # truncation=True: 如果文本太长就截断
        # max_length=512: 最大输入长度512个token
        inputs = self.tokenizer(formatted_prompt, return_tensors="pt", truncation=True, max_length=512)
        
        # 将输入数据移动到当前GPU上
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        # 生成回答
        with torch.no_grad():  # 不计算梯度，节省内存
            outputs = self.model.generate(
                **inputs, 
                max_new_tokens=512,                        # 最多生成512个新token
                temperature=0.7,                           # 温度参数，控制生成的随机性
                top_p=0.9,                                # 核采样参数
                do_sample=True,                           # 启用采样生成
                pad_token_id=self.tokenizer.eos_token_id  # 填充token ID
            )
        
        # 解码生成的回答
        # 只取新生成的部分，跳过原始输入
        response = self.tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True).strip()
        return response
    
    def compute_reward(self, prompt: str, completion: str) -> float:
        """
        计算奖励分数
        作用：使用奖励模型给问题-回答对打分
        prompt: 问题文本
        completion: 回答文本
        返回：奖励分数（浮点数）
        """
        try:
            # 使用与奖励模型训练完全相同的LLaMA-3格式
            # 这个格式化非常重要，必须与训练时保持一致
            formatted_text = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{completion}<|eot_id|>"
            
            # 将格式化的文本转换为模型输入
            inputs = self.reward_tokenizer(
                formatted_text, 
                truncation=True,      # 截断过长文本
                padding=True,         # 填充到相同长度
                max_length=2048,      # 最大长度2048
                return_tensors="pt"   # 返回PyTorch张量
            )
            
            # 将输入移动到当前GPU
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            
            # 计算奖励分数
            with torch.no_grad():
                outputs = self.reward_model(**inputs)
                # 提取分数并转换为Python数值
                reward = outputs.logits.squeeze().cpu().item()
            
            return reward
        except Exception as e:
            # 如果计算失败，只在主进程打印错误信息
            if self.local_rank == 0:
                print(f"奖励计算失败: {e}")
            return 0.0
    
    def evaluate_batch(self, prompts: list) -> list:
        """
        批量评测
        作用：对一批问题进行评测，每个GPU处理不同的问题
        prompts: 问题列表
        返回：评测结果列表
        """
        results = []
        
        # 创建数据集和数据加载器
        # 每个问题会生成4个回答
        dataset = PromptDataset(prompts, num_generations=4)
        dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
        
        # 分布式处理：每个GPU处理不同的样本
        for i, batch_prompts in enumerate(tqdm(dataloader, desc=f"GPU-{self.local_rank}", disable=self.local_rank!=0)):
            prompt = batch_prompts[0]
            
            # 工作分配：第i个样本由第(i % world_size)个GPU处理
            # 例如：GPU0处理样本0,3,6..., GPU1处理样本1,4,7...
            if i % self.world_size != self.local_rank:
                continue
            
            # 生成回复和计算奖励
            response = self.generate_response(prompt)
            reward = self.compute_reward(prompt, response)
            
            # 保存结果
            results.append({
                'prompt': prompt,
                'response': response,
                'reward': reward,
                'gpu_id': self.local_rank
            })
        
        return results
    
    def gather_results(self, local_results: list) -> list:
        """
        收集所有GPU的结果
        作用：将各个GPU的评测结果汇总，计算每个问题的平均分数
        local_results: 当前GPU的评测结果
        返回：汇总后的最终结果（只在主进程返回）
        """
        # 简单的结果收集 - 在实际使用中可以用torch.distributed.all_gather
        # 这里为了简化，直接使用本地结果
        all_results = local_results
        
        # 只在主进程（GPU 0）进行结果汇总
        if self.local_rank == 0:
            # 按问题分组
            prompt_groups = {}
            for result in all_results:
                prompt = result['prompt']
                if prompt not in prompt_groups:
                    prompt_groups[prompt] = []
                prompt_groups[prompt].append(result)
            
            # 计算每个问题的统计信息
            final_results = []
            for prompt, group in prompt_groups.items():
                # 提取所有回答的奖励分数
                rewards = [r['reward'] for r in group]
                responses = [r['response'] for r in group]
                
                # 计算统计指标
                final_results.append({
                    'prompt': prompt,
                    'responses': responses,           # 所有生成的回答
                    'rewards': rewards,               # 所有奖励分数
                    'mean_reward': sum(rewards) / len(rewards),  # 平均奖励
                    'max_reward': max(rewards),       # 最高奖励
                    'min_reward': min(rewards)        # 最低奖励
                })
            
            return final_results
        
        return []

def load_prompts_from_jsonl(file_path: str, sample_size: int = None) -> list:
    """
    从JSONL文件加载问题
    作用：读取训练数据文件，提取其中的问题部分
    file_path: JSONL文件路径
    sample_size: 采样数量，如果为None则加载全部
    返回：问题列表
    """
    prompts = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data = json.loads(line)
                # 处理两种数据格式
                if isinstance(data.get('prompt'), list):
                    # 如果prompt是消息列表格式，提取用户消息
                    user_msg = next(msg['content'] for msg in data['prompt'] if msg['role'] == 'user')
                    prompts.append(user_msg)
                else:
                    # 如果prompt是字符串格式，直接使用
                    prompts.append(data['prompt'])
    
    # 如果指定了采样数量，只取前sample_size个
    if sample_size:
        prompts = prompts[:sample_size]
    
    return prompts

def main():
    """
    主函数
    作用：解析命令行参数，运行多GPU评测流程
    """
    # 设置命令行参数
    parser = argparse.ArgumentParser(description="多机多卡奖励模型评分工具")
    parser.add_argument('--model', type=str, required=True, help='模型路径')
    parser.add_argument('--reward_model', type=str, required=True, help='奖励模型路径')
    parser.add_argument('--prompts', type=str, help='prompts文件路径')
    parser.add_argument('--sample_size', type=int, default=20, help='评测样本数量')
    parser.add_argument('--local_rank', type=int, default=0, help='本地rank')
    
    args = parser.parse_args()
    
    # 创建多GPU评分器
    evaluator = MultiGPURewardEvaluator(args.model, args.reward_model)
    
    # 加载问题数据
    if args.prompts:
        # 从文件加载问题
        prompts = load_prompts_from_jsonl(args.prompts, args.sample_size)
    else:
        # 使用默认问题
        prompts = ["请分析一下当前股市的投资机会和风险。"] * 5
    
    # 只在主进程打印开始信息
    if evaluator.local_rank == 0:
        print(f"开始多GPU评测 {len(prompts)} 个prompts...")
    
    # 运行评测流程
    local_results = evaluator.evaluate_batch(prompts)      # 各GPU并行评测
    final_results = evaluator.gather_results(local_results)  # 汇总结果
    
    # 主进程输出和保存结果
    if evaluator.local_rank == 0 and final_results:
        # 计算整体统计信息
        all_rewards = [r['mean_reward'] for r in final_results]
        print(f"\n多GPU评测结果:")
        print(f"总prompts数: {len(final_results)}")
        print(f"平均奖励: {sum(all_rewards)/len(all_rewards):.4f}")
        print(f"最高奖励: {max(all_rewards):.4f}")
        print(f"最低奖励: {min(all_rewards):.4f}")
        
        # 保存详细结果到JSON文件
        output_file = f"multigpu_evaluation_{args.model.split('/')[-1]}.json"
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(final_results, f, ensure_ascii=False, indent=2)
        print(f"结果已保存到: {output_file}")

if __name__ == "__main__":
    main()
```

run_evaluation.sh   评估启动脚本

```bash
#!/bin/bash

# 多机多卡评测脚本

# 配置参数
# MODEL_PATH="./output/best_model"
MODEL_PATH="/shared/DeepSeek-R1-Distill-Qwen-7B_028/best_complete_model_05261653_028"
REWARD_MODEL_PATH="/shared/Skywork-Reward_checkpoint-1000/checkpoint-1000"
PROMPTS_PATH="/shared/grpo_financial_tuning/data/test_prompts_dataset.jsonl"
SAMPLE_SIZE=50

# 多机多卡配置
declare -A NODES=(
    ["10.60.68.220"]=1    # 主节点：2张GPU
    ["10.60.98.173"]=1    # 从节点：1张GPU
)
MASTER_ADDR="10.60.68.220"

# 创建hostfile
create_hostfile() {
    echo "创建DeepSpeed hostfile..."
    > ./hostfile
    for node_ip in "${!NODES[@]}"; do
        gpu_count=${NODES[$node_ip]}
        echo "$node_ip slots=$gpu_count" >> ./hostfile
    done
    cat ./hostfile
}

# 单机多卡模式
single_node() {
    echo "启动单机多卡评测..."
    deepspeed --num_gpus=2 model_eval/multigpu_evaluator.py \
        --model $MODEL_PATH \
        --reward_model $REWARD_MODEL_PATH \
        --prompts $PROMPTS_PATH \
        --sample_size $SAMPLE_SIZE
}

# 多机多卡模式
multi_node() {
    echo "启动多机多卡评测..."
    create_hostfile
    
    deepspeed --hostfile=hostfile model_eval/multigpu_evaluator.py \
        --model $MODEL_PATH \
        --reward_model $REWARD_MODEL_PATH \
        --prompts $PROMPTS_PATH \
        --sample_size $SAMPLE_SIZE
}

# 解析参数
case $1 in
    --single)
        single_node
        ;;
    --multi)
        multi_node
        ;;
    *)
        echo "用法: $0 [--single|--multi]"
        echo "  --single: 单机多卡模式"
        echo "  --multi:  多机多卡模式"
        exit 1
        ;;
esac
```

执行命令 ./run_evaluation.sh --multi

GRPO微调后模型效果得分

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250707122127283.png" width=100%></div>

GRPO微调前模型效果得分

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250707122153839.png" width=100%></div>

# 7. 模型分布式部署

```bash
# 使用清华源加速
pip install -i https://pypi.tuna.tsinghua.edu.cn/simple vllm
pip install -i https://pypi.tuna.tsinghua.edu.cn/simple ray[default]
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250707171539903.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250707171601452.png" width=100%></div>

sudo ufw allow 22

其余端口如果有限制命令相同例如 sudo ufw allow 29500

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250707180554224.png" width=100%></div>

deploy_A800.sh A800部署脚本

```bash
#!/bin/bash

# ==================== 项目部署脚本 ====================
# 作用：将本地项目文件同步到远程服务器，用于A800 GPU服务器部署    执行时删除注释使用 执行时删除注释使用
# 使用方法：./deploy_A800.sh

echo "🚀 开始同步项目到服务器..."

# 使用rsync命令进行文件同步
# -a：归档模式，保持文件属性
# -v：详细输出，显示同步过程
# -z：压缩传输，节省带宽
# --exclude：排除不需要同步的文件和目录
rsync -avz \
  --exclude '.git' \         # 排除Git版本控制文件
  --exclude '__pycache__' \  # 排除Python编译缓存文件
  --exclude '.DS_Store' \    # 排除macOS系统文件
  ./ \                       # 当前目录作为源目录
  ubuntu@117.50.194.2:/shared/grpo_financial_tuning/  # 目标服务器路径

echo "✅ 文件同步完成，执行远程操作..."

# 通过SSH连接到远程服务器并执行命令
# << 'EOF' 表示开始多行命令输入，直到遇到EOF结束
ssh ubuntu@117.50.194.2 << 'EOF'
cd /shared/grpo_financial_tuning    # 切换到项目目录
chmod +x run_training.sh           # 给训练脚本添加执行权限
chmod +x run_evaluation.sh         # 给评估脚本添加执行权限
chmod +x vllm_run.sh               # 给vLLM运行脚本添加执行权限
# 如果有需要重启的服务或运行命令，请在下面加
# 示例: pkill -f app.py && nohup python3 app.py > out.log 2>&1 &
echo "🚀 已进入远程项目目录，可根据需要添加执行命令"
EOF

echo "🎉 部署完毕！"
```

vllm_run.sh 部署脚本

```bash
#!/bin/bash

## 脚本功能解析

### 整体架构
这个脚本实现了一个**分布式模型推理服务**，具体架构为：
- **头节点（10.60.68.220）**：运行vLLM API服务器，提供1个GPU
- **工作节点（10.60.98.173）**：提供额外的1个GPU算力
- **总算力**：2个GPU通过张量并行协同工作

### 关键技术组件

1. **Ray集群管理**：
   - 负责多机资源调度和任务分配
   - 头节点协调，工作节点提供算力
   - 通过端口6379进行集群通信

2. **vLLM推理引擎**：
   - 高性能大语言模型推理框架
   - 支持张量并行，将模型分割到多个GPU
   - 提供OpenAI兼容的API接口

3. **张量并行（tensor-parallel-size 2）**：
   - 将7B模型的参数分割到2个GPU上
   - 每个GPU处理模型的一部分，协同完成推理
   - 相比单GPU，推理速度提升约1.8倍

### 资源配置说明

- **显存使用**：每个GPU只使用30%显存，7B模型约需要6-8GB显存
- **网络通信**：强制使用eth0网卡，确保稳定的节点间通信
- **API访问**：通过8000端口提供服务，需要API密钥认证

### 使用场景
这个配置适合**生产环境的模型部署**，可以：
- 同时服务多个客户端请求
- 提供稳定的推理性能
- 支持高并发访问
- 便于集成到现有系统中

# 定义集群节点IP地址
# 这些IP地址应该根据实际的集群配置进行修改
HEAD_NODE="10.60.68.220"    # 头节点IP地址 - 主服务器，负责协调和管理整个集群
WORKER_NODE="10.60.98.173" # 工作节点IP地址 - 从服务器，提供额外的GPU算力
# MODEL_PATH="/shared/DeepSeek-R1-Distill-Qwen-7B_028/best_complete_model_05261653_028"
MODEL_PATH="/shared/grpo_financial_tuning/output/best_model"  # 模型文件路径 - GRPO训练完成后保存的最佳模型
PORT=8000  # API服务端口号 - 客户端通过这个端口访问模型服务

# 启动Ray集群
# Ray是一个分布式计算框架，用于管理多机多卡的资源调度
echo "启动Ray集群"

# 激活conda环境
# conda是Python包管理工具，这里激活名为reward3的虚拟环境
# 虚拟环境包含了运行vLLM所需的所有依赖包
source ~/miniconda3/etc/profile.d/conda.sh
conda activate reward3

# 设置PyTorch分布式环境变量
# 这些环境变量告诉PyTorch如何在多机之间进行通信
export MASTER_ADDR=$HEAD_NODE  # 主节点地址 - 分布式训练的协调节点
export MASTER_PORT=29500       # 主节点端口 - 用于节点间通信的端口
export GLOO_SOCKET_IFNAME=eth0  # 指定网卡接口 - 强制使用eth0网卡进行通信
export NCCL_SOCKET_IFNAME=eth0  # NCCL通信网卡 - NVIDIA集合通信库使用的网卡

# 强制停止现有的Ray进程，然后启动头节点
# 2>/dev/null: 将错误输出重定向到空设备，避免显示错误信息
# || true: 即使命令失败也继续执行，不会中断脚本
ray stop --force 2>/dev/null || true

# 启动Ray头节点
# --head: 指定这是头节点（主节点）
# --port=6379: Ray集群通信端口
# --dashboard-host=0.0.0.0: 允许从任何IP访问Ray控制面板
# --dashboard-port=8265: Ray控制面板的Web端口
# --num-gpus=1: 头节点提供1个GPU资源
# --num-cpus=24: 头节点提供24个CPU核心
ray start --head --port=6379 --dashboard-host=0.0.0.0 --dashboard-port=8265 --num-gpus=1 --num-cpus=24

# 启动工作节点
echo "连接工作节点"

# 通过SSH连接到工作节点并启动Ray worker
# SSH是安全外壳协议，用于远程登录和执行命令
# ubuntu@$WORKER_NODE: 以ubuntu用户身份连接到工作节点
ssh ubuntu@$WORKER_NODE "
    # 在工作节点上激活相同的conda环境
    source ~/miniconda3/etc/profile.d/conda.sh
    conda activate reward3
    
    # 停止工作节点上可能存在的Ray进程
    ray stop --force 2>/dev/null || true
    
    # 启动Ray工作节点
    # --address='$HEAD_NODE:6379': 连接到头节点的Ray集群
    # --num-gpus=1: 工作节点提供1个GPU资源
    ray start --address='$HEAD_NODE:6379' --num-gpus=1
" 2>/dev/null || echo "工作节点连接失败，继续单节点运行"
# 如果SSH连接失败，会输出提示信息但不会中断脚本执行

echo "🚀 启动vLLM多机多卡服务器..."

# 启动vLLM API服务器
# vLLM是一个高性能的大语言模型推理引擎
python -m vllm.entrypoints.openai.api_server \
  --model $MODEL_PATH \                    # 指定要加载的模型路径
  --host 0.0.0.0 \                        # 服务器监听地址，0.0.0.0表示接受来自任何IP的请求
  --port $PORT \                          # 服务器端口号，客户端通过这个端口访问API
  --tensor-parallel-size 2 \              # 张量并行大小，将模型分割到2个GPU上并行计算
  --dtype bfloat16 \                      # 数据类型，使用bfloat16精度节省显存
  --max-model-len 2048 \                  # 最大序列长度，限制输入+输出的总token数
  --gpu-memory-utilization 0.3 \         # GPU显存利用率，只使用30%的显存，为其他进程预留空间
  --disable-log-requests \                # 禁用请求日志，减少日志输出
  --api-key grpo-model-key \              # API密钥，客户端需要提供这个密钥才能访问服务
  --distributed-executor-backend ray      # 分布式执行后端，使用Ray框架管理多机多卡资源
```

执行命令 ./vllm_run.sh 

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250707201954623.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250707201851108.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250707201930558.png" width=100%></div>

请求测试命令

```python
python -c "
import requests
resp = requests.post('http://localhost:8000/v1/chat/completions', 
    headers={'Authorization': 'Bearer grpo-model-key'},
    json={'model': '/shared/DeepSeek-R1-Distill-Qwen-7B_028/best_complete_model_05261653_028', 'messages': [{'role': 'user', 'content': '你是一个专业的金融领域分析师请对以下问题进行详细解答：周三盘前交易时段，金融板块呈现温和上涨态势：金融精选行业SPDR基金(XLF)上涨0.4%，Direxion每日三倍做多金融股ETF(FAS)上涨1.2%，而其反向产品Direxion每日三倍做空金融股ETF(FAZ)下跌1.1%。万事达卡(MA)股价上涨0.7%，因其董事会批准了最高110亿美元的普通股回购计划，并将季度股息提高16%。Cboe全球市场(CBOE)股价上涨0.5%，该公司披露其指数期权合约的11月日均交易量达到近400万份，较去年同期的330万份增长22%。基于这些市场动态，请分析以下问题：在当前市场环境下，影响金融板块ETF(XLF、FAS、FAZ)价格波动的关键驱动因素有哪些？如何评估万事达卡大规模股票回购和股息增长对其长期估值的影响？Cboe交易量的大幅增长可能预示着哪些市场结构变化？请综合考虑宏观经济环境、行业竞争格局和公司特定事件等多重因素展开分析。'}], 'max_tokens': 1024})
print('Status Code:', resp.status_code)
print('Response:', resp.json())
"
python -c "
import requests
resp = requests.post('http://localhost:8000/v1/chat/completions', 
    headers={'Authorization': 'Bearer grpo-model-key'},
    json={'model': '/shared/grpo_financial_tuning/output/best_model', 'messages': [{'role': 'user', 'content': '你是一个专业的金融领域分析师请对以下问题进行详细解答：周三盘前交易时段，金融板块呈现温和上涨态势：金融精选行业SPDR基金(XLF)上涨0.4%，Direxion每日三倍做多金融股ETF(FAS)上涨1.2%，而其反向产品Direxion每日三倍做空金融股ETF(FAZ)下跌1.1%。万事达卡(MA)股价上涨0.7%，因其董事会批准了最高110亿美元的普通股回购计划，并将季度股息提高16%。Cboe全球市场(CBOE)股价上涨0.5%，该公司披露其指数期权合约的11月日均交易量达到近400万份，较去年同期的330万份增长22%。基于这些市场动态，请分析以下问题：在当前市场环境下，影响金融板块ETF(XLF、FAS、FAZ)价格波动的关键驱动因素有哪些？如何评估万事达卡大规模股票回购和股息增长对其长期估值的影响？Cboe交易量的大幅增长可能预示着哪些市场结构变化？请综合考虑宏观经济环境、行业竞争格局和公司特定事件等多重因素展开分析。'}], 'max_tokens': 1024})
print('Status Code:', resp.status_code)
print('Response:', resp.json())
"
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250709154443396.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250707201838482.png" width=100%></div>

# 8. 总结

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250709164109046.png" width=100%></div>